
---

# **Chapter 16: Navigation Fundamentals**

---

## **Learning Objectives**

By the end of this chapter, you will be able to:

- Understand the Navigator and Route classes in Flutter
- Implement push, pop, and pushReplacement navigation patterns
- Pass data between screens using constructors and named routes
- Configure and use named routes with a route map
- Create dynamic routing with onGenerateRoute
- Handle back button behavior and navigation stack management
- Implement common navigation patterns (bottom navigation, tabs, drawers)

---

## **Prerequisites**

- Completed Chapter 7: Widget Fundamentals
- Completed Chapter 8: Layout and Composition
- Understanding of StatefulWidget and StatelessWidget
- Basic familiarity with Flutter widget tree structure
- Knowledge of BuildContext and its usage

---

## **16.1 Navigator and Route Classes**

Navigation in Flutter is managed by the `Navigator` widget, which maintains a stack of `Route` objects. Each route represents a screen or page in your app.

### **Understanding the Navigator Widget**

The `Navigator` is a widget that manages a stack of `Route` objects. It provides methods to push routes onto the stack (navigate to new screens) and pop routes off the stack (go back to previous screens).

```dart
import 'package:flutter/material.dart';

void main() {
  runApp(const NavigationBasicsApp());
}

class NavigationBasicsApp extends StatelessWidget {
  const NavigationBasicsApp({super.key});

  @override
  Widget build(BuildContext context) {
    return MaterialApp(
      title: 'Navigation Basics',
      // MaterialApp automatically includes a Navigator
      // The Navigator is positioned at the root of the widget tree
      // It manages the route stack for the entire app
      home: const FirstScreen(),
    );
  }
}

class FirstScreen extends StatelessWidget {
  const FirstScreen({super.key});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: const Text('First Screen'),
      ),
      body: Center(
        child: ElevatedButton(
          onPressed: () {
            // Navigator.of(context) gets the Navigator widget nearest in the widget tree
            // context is the BuildContext provided by the build method
            // The Navigator manages a stack of Route objects (screens)
            Navigator.of(context).push(
              MaterialPageRoute(
                // MaterialPageRoute defines a route that transitions to a new screen
                // It uses a platform-specific transition (slide right on iOS, slide up on Android)
                builder: (context) {
                  // builder is a function that returns the widget for the new route
                  // It receives the BuildContext for the new route
                  return const SecondScreen();
                },
              ),
            );
          },
          child: const Text('Go to Second Screen'),
        ),
      ),
    );
  }
}

class SecondScreen extends StatelessWidget {
  const SecondScreen({super.key});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: const Text('Second Screen'),
      ),
      body: Center(
        child: ElevatedButton(
          onPressed: () {
            // Navigator.pop removes the current route from the stack
            // This returns the user to the previous screen
            // The optional result parameter can pass data back to the previous screen
            Navigator.of(context).pop();
          },
          child: const Text('Go Back'),
        ),
      ),
    );
  }
}
```

**Explanation:**

- **`MaterialApp`**: The `MaterialApp` widget automatically includes a `Navigator` at the root of the widget tree. This `Navigator` manages the entire app's navigation stack. You don't need to create a `Navigator` explicitly when using `MaterialApp` or `CupertinoApp`.
- **`Navigator.of(context)`**: Retrieves the `Navigator` widget that is the nearest ancestor in the widget tree. This is the idiomatic way to get the `Navigator` in Flutter. The `context` parameter is the `BuildContext` from the current widget's `build` method.
- **`Navigator.push()`**: Adds a new route to the top of the navigation stack. This causes the new screen to appear. The method returns a `Future` that completes when the route is popped off the stack, allowing you to receive a result from the navigated-to screen.
- **`MaterialPageRoute`**: A route that uses a platform-appropriate transition. On iOS, it slides the new screen from right to left. On Android, it slides the new screen from bottom to top. It also handles system gestures (like the iOS swipe-to-go-back gesture).
- **`builder` parameter**: A function that builds the widget tree for the new route. It receives a `BuildContext` that is scoped to the new route, allowing the built widget to access navigation methods via `Navigator.of(context)`.
- **`Navigator.pop()`**: Removes the topmost route from the navigation stack, effectively navigating back to the previous screen. You can optionally pass a result back to the previous screen by providing a parameter to `pop()`.
- **Navigation stack**: The `Navigator` maintains a stack of `Route` objects. Pushing adds a route to the top of the stack; popping removes the top route. The bottommost route is the initial route (the home screen).

### **Route Classes Explained**

Flutter provides several built-in route classes for different navigation patterns.

```dart
import 'package:flutter/material.dart';

void main() {
  runApp(const RouteTypesApp());
}

class RouteTypesApp extends StatelessWidget {
  const RouteTypesApp({super.key});

  @override
  Widget build(BuildContext context) {
    return MaterialApp(
      title: 'Route Types',
      home: const RouteTypesScreen(),
    );
  }
}

class RouteTypesScreen extends StatelessWidget {
  const RouteTypesScreen({super.key});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: const Text('Route Types'),
      ),
      body: ListView(
        children: [
          ListTile(
            title: const Text('MaterialPageRoute'),
            subtitle: const Text('Platform-specific transition'),
            trailing: const Icon(Icons.arrow_forward),
            onTap: () {
              // MaterialPageRoute uses a platform-appropriate transition
              // On iOS: slides from right
              // On Android: slides from bottom
              Navigator.of(context).push(
                MaterialPageRoute(
                  builder: (context) => const DetailScreen('MaterialPageRoute'),
                ),
              );
            },
          ),
          ListTile(
            title: const Text('CupertinoPageRoute'),
            subtitle: const Text('iOS-style transition'),
            trailing: const Icon(Icons.arrow_forward),
            onTap: () {
              // CupertinoPageRoute always uses the iOS-style slide transition
              // It also includes the iOS back swipe gesture
              Navigator.of(context).push(
                CupertinoPageRoute(
                  builder: (context) => const DetailScreen('CupertinoPageRoute'),
                ),
              );
            },
          ),
          ListTile(
            title: const Text('PageRouteBuilder'),
            subtitle: const Text('Custom transition'),
            trailing: const Icon(Icons.arrow_forward),
            onTap: () {
              // PageRouteBuilder allows complete customization of the transition
              // You can define your own animation curves, durations, and transitions
              Navigator.of(context).push(
                PageRouteBuilder(
                  // pageBuilder builds the content of the route
                  pageBuilder: (context, animation, secondaryAnimation) {
                    return const DetailScreen('PageRouteBuilder');
                  },
                  // transitionDuration defines how long the transition takes
                  transitionDuration: const Duration(milliseconds: 500),
                  // reverseTransitionDuration defines the duration for going back
                  reverseTransitionDuration: const Duration(milliseconds: 300),
                  // transitionsBuilder defines the visual transition
                  transitionsBuilder: (context, animation, secondaryAnimation, child) {
                    // animation is the animation for the forward navigation (pushing)
                    // secondaryAnimation is for the background route's animation
                    // child is the widget returned by pageBuilder
                    
                    // Using FadeTransition: opacity goes from 0 to 1
                    return FadeTransition(
                      opacity: animation,
                      child: child,
                    );
                    
                    // Alternative: SlideTransition from bottom
                    // return SlideTransition(
                    //   position: Tween<Offset>(
                    //     begin: const Offset(0.0, 1.0), // Start at bottom
                    //     end: Offset.zero, // End at normal position
                    //   ).animate(CurvedAnimation(
                    //     parent: animation,
                    //     curve: Curves.easeInOut,
                    //   )),
                    //   child: child,
                    // );
                    
                    // Alternative: ScaleTransition
                    // return ScaleTransition(
                    //   scale: animation,
                    //   child: child,
                    // );
                  },
                ),
              );
            },
          ),
          ListTile(
            title: const Text('FullscreenDialog'),
            subtitle: const Text('Dialog-style page'),
            trailing: const Icon(Icons.arrow_forward),
            onTap: () {
              // fullscreenDialog makes the page appear as a modal dialog
              // On iOS, it slides from the bottom instead of the right
              // On Android, it also has a different appearance
              Navigator.of(context).push(
                MaterialPageRoute(
                  builder: (context) => const DetailScreen('FullscreenDialog'),
                  fullscreenDialog: true,
                ),
              );
            },
          ),
        ],
      ),
    );
  }
}

class DetailScreen extends StatelessWidget {
  final String routeType;
  
  const DetailScreen(this.routeType, {super.key});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: Text(routeType),
      ),
      body: Center(
        child: Column(
          mainAxisAlignment: MainAxisAlignment.center,
          children: [
            Text('Navigated using $routeType'),
            const SizedBox(height: 20),
            ElevatedButton(
              onPressed: () => Navigator.of(context).pop(),
              child: const Text('Go Back'),
            ),
          ],
        ),
      ),
    );
  }
}
```

**Explanation:**

- **`MaterialPageRoute`**: The most commonly used route type. It automatically selects the appropriate transition based on the platform (iOS or Android). It handles the system back button on Android and the swipe gesture on iOS, providing a native feel.
- **`CupertinoPageRoute`**: Always uses the iOS-style transition, regardless of the platform. This is useful when you want a consistent iOS look across all platforms or when building iOS-specific screens in a cross-platform app.
- **`PageRouteBuilder`**: Provides complete control over the navigation transition. You can:
  - Define custom animations using the `animation` parameter
  - Specify transition durations separately for forward and reverse navigation
  - Use multiple transitions together (e.g., fade and slide)
  - Create complex, custom transitions that don't exist in the built-in route types
- **`transitionsBuilder` parameters**:
  - `context`: The build context for the route
  - `animation`: An `Animation<double>` that goes from 0.0 to 1.0 during the forward navigation
  - `secondaryAnimation`: An animation for the previous route's transition (used when you want to animate the outgoing route)
  - `child`: The widget built by `pageBuilder` that should be displayed
- **`fullscreenDialog`**: When set to `true`, the route appears as a modal dialog instead of a full page. On iOS, this changes the direction of the slide animation (from bottom instead of from right) and adds a "Cancel" button in the navigation bar. On Android, it changes the visual appearance to look more like a dialog.
- **Common transitions**:
  - `FadeTransition`: Animates opacity from 0 to 1
  - `SlideTransition`: Animates position using an `Offset`
  - `ScaleTransition`: Animates scale (size) from 0 to 1
  - `RotationTransition`: Animates rotation
  - `SizeTransition`: Animates height (useful for accordions)

---

## **16.2 Push, Pop, and PushReplacement**

Flutter provides several methods for navigating between screens, each with different behaviors and use cases.

### **Basic Push and Pop**

```dart
import 'package:flutter/material.dart';

void main() {
  runApp(const PushPopApp());
}

class PushPopApp extends StatelessWidget {
  const PushPopApp({super.key});

  @override
  Widget build(BuildContext context) {
    return MaterialApp(
      title: 'Push and Pop',
      home: const HomeScreen(),
    );
  }
}

class HomeScreen extends StatelessWidget {
  const HomeScreen({super.key});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: const Text('Home Screen'),
      ),
      body: Center(
        child: ElevatedButton(
          onPressed: () {
            // Navigator.push adds a new route to the navigation stack
            // The user will see the new screen and can go back
            Navigator.push(
              context,
              MaterialPageRoute(
                builder: (context) => const ScreenA(),
              ),
            );
          },
          child: const Text('Go to Screen A'),
        ),
      ),
    );
  }
}

class ScreenA extends StatelessWidget {
  const ScreenA({super.key});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: const Text('Screen A'),
      ),
      body: Center(
        child: Column(
          mainAxisAlignment: MainAxisAlignment.center,
          children: [
            const Text('This is Screen A'),
            const SizedBox(height: 20),
            ElevatedButton(
              onPressed: () {
                // Navigate to Screen B while keeping Screen A in the stack
                // The navigation stack will be: [Home, A, B]
                Navigator.push(
                  context,
                  MaterialPageRoute(
                    builder: (context) => const ScreenB(),
                  ),
                );
              },
              child: const Text('Go to Screen B'),
            ),
            const SizedBox(height: 20),
            ElevatedButton(
              onPressed: () {
                // Navigator.pop removes the current route from the stack
                // This will return the user to the Home screen
                // The navigation stack will be: [Home]
                Navigator.pop(context);
              },
              child: const Text('Go Back'),
            ),
          ],
        ),
      ),
    );
  }
}

class ScreenB extends StatelessWidget {
  const ScreenB({super.key});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: const Text('Screen B'),
      ),
      body: Center(
        child: Column(
          mainAxisAlignment: MainAxisAlignment.center,
          children: [
            const Text('This is Screen B'),
            const SizedBox(height: 20),
            ElevatedButton(
              onPressed: () {
                // pop() without arguments returns null
                Navigator.pop(context);
              },
              child: const Text('Go Back (No Result)'),
            ),
            const SizedBox(height: 20),
            ElevatedButton(
              onPressed: () {
                // pop() with an argument passes data back to the previous screen
                // The previous screen can receive this data using the Future from push()
                Navigator.pop(context, 'Result from Screen B');
              },
              child: const Text('Go Back with Result'),
            ),
          ],
        ),
      ),
    );
  }
}
```

**Explanation:**

- **`Navigator.push()`**: Adds a new route to the top of the navigation stack. This is the most common navigation method. The method returns a `Future` that completes when the pushed route is popped. If `pop()` is called with an argument, that argument becomes the value of the `Future`.
- **Navigation stack visualization**: Each call to `push()` adds a new route to the stack. Starting from `[Home]`, after pushing to Screen A: `[Home, A]`. After pushing to Screen B: `[Home, A, B]`.
- **`Navigator.pop()`**: Removes the topmost route from the navigation stack. This is equivalent to pressing the back button (on Android) or swiping back (on iOS). After calling `pop()` from Screen B: `[Home, A]`.
- **`pop()` with a result**: When you call `Navigator.pop(context, value)`, the `value` is returned to the previous screen. The previous screen can capture this value by awaiting the `Future` returned from its `push()` call.
- **Multiple pops**: You can call `pop()` multiple times in succession, or use `Navigator.popUntil()` to pop multiple routes at once (covered later in this chapter).
- **System behavior**: The hardware back button on Android and the swipe-back gesture on iOS both call `pop()` under the hood. Your custom `pop()` calls behave identically to these system actions.

### **Receiving Results from Navigation**

```dart
import 'package:flutter/material.dart';

void main() {
  runApp(const ResultNavigationApp());
}

class ResultNavigationApp extends StatelessWidget {
  const ResultNavigationApp({super.key});

  @override
  Widget build(BuildContext context) {
    return MaterialApp(
      title: 'Result Navigation',
      home: const SelectionScreen(),
    );
  }
}

class SelectionScreen extends StatefulWidget {
  const SelectionScreen({super.key});

  @override
  State<SelectionScreen> createState() => _SelectionScreenState();
}

class _SelectionScreenState extends State<SelectionScreen> {
  String? selectedOption;

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: const Text('Selection Screen'),
      ),
      body: Center(
        child: Column(
          mainAxisAlignment: MainAxisAlignment.center,
          children: [
            Text(
              selectedOption ?? 'No option selected',
              style: const TextStyle(fontSize: 20),
            ),
            const SizedBox(height: 20),
            ElevatedButton(
              onPressed: () async {
                // The push method returns a Future that completes when the route is popped
                // The Future's value is the result passed to pop()
                // We use 'await' to wait for the result before continuing
                
                final result = await Navigator.push(
                  context,
                  MaterialPageRoute(
                    builder: (context) => const OptionsScreen(),
                  ),
                );
                
                // When the user pops the OptionsScreen, control resumes here
                // result contains the value passed to Navigator.pop() in OptionsScreen
                // If pop() was called without an argument, result is null
                
                setState(() {
                  selectedOption = result?.toString() ?? 'None';
                });
              },
              child: const Text('Select an Option'),
            ),
          ],
        ),
      ),
    );
  }
}

class OptionsScreen extends StatelessWidget {
  const OptionsScreen({super.key});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: const Text('Options'),
      ),
      body: ListView(
        children: [
          ListTile(
            title: const Text('Option A'),
            onTap: () {
              // Pop with a result
              // This returns 'Option A' to the SelectionScreen
              // The Future in SelectionScreen completes with this value
              Navigator.pop(context, 'Option A');
            },
          ),
          ListTile(
            title: const Text('Option B'),
            onTap: () {
              Navigator.pop(context, 'Option B');
            },
          ),
          ListTile(
            title: const Text('Option C'),
            onTap: () {
              Navigator.pop(context, 'Option C');
            },
          ),
          ListTile(
            title: const Text('Cancel'),
            onTap: () {
              // Pop without a result (returns null)
              Navigator.pop(context);
            },
          ),
        ],
      ),
    );
  }
}
```

**Explanation:**

- **`await` with `push()`**: The `Navigator.push()` method returns a `Future<dynamic>`. Using `await`, you can pause execution until the route is popped and the result is available. This is the primary pattern for receiving data from a navigated-to screen.
- **Result data type**: The result can be of any type (`String`, `int`, `Object`, custom class, etc.). The type is `dynamic` in the method signature, so you may need to cast or check the type depending on your use case.
- **`null` results**: If `pop()` is called without an argument, or if the user uses the system back button, the `Future` completes with `null`. Always handle the `null` case in your code, as shown with `result?.toString() ?? 'None'`.
- **Multiple results**: Each `push()` call creates a separate `Future`. If you navigate Screen A → Screen B → Screen C, and Screen C pops with a result, only Screen B's `Future` completes. Screen A's `Future` won't complete until Screen B is popped (which may or may not pass along the result).
- **Alternative pattern**: Instead of using `await`, you can use `.then()` on the `Future`:
  ```dart
  Navigator.push(...).then((result) {
    setState(() {
      selectedOption = result?.toString() ?? 'None';
    });
  });
  ```
  This is useful when you don't want to make the method `async` or when you want to handle the result independently of other async operations.

### **PushReplacement**

`pushReplacement` replaces the current route with a new route, removing the current route from the stack.

```dart
import 'package:flutter/material.dart';

void main() {
  runApp(const PushReplacementApp());
}

class PushReplacementApp extends StatelessWidget {
  const PushReplacementApp({super.key});

  @override
  Widget build(BuildContext context) {
    return MaterialApp(
      title: 'Push Replacement',
      // initialRoute sets the starting route when using named routes
      initialRoute: '/',
      routes: {
        '/': (context) => const LoginScreen(),
        '/home': (context) => const HomeScreen(),
        '/profile': (context) => const ProfileScreen(),
      },
    );
  }
}

class LoginScreen extends StatelessWidget {
  const LoginScreen({super.key});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: const Text('Login'),
        // automaticallyImplyLeading: false removes the back button
        // This is useful for login screens where the user shouldn't go back
        automaticallyImplyLeading: false,
      ),
      body: Center(
        child: ElevatedButton(
          onPressed: () {
            // pushReplacement removes the current route and adds a new route
            // After this, the navigation stack is: [/home]
            // The / route (LoginScreen) is removed
            // User cannot go back to the login screen using the back button
            
            Navigator.pushReplacement(
              context,
              MaterialPageRoute(
                builder: (context) => const HomeScreen(),
                // You can also set a name for the route when using pushReplacement
                // This can be useful for debugging or analytics
                settings: const RouteSettings(name: '/home'),
              ),
            );
          },
          child: const Text('Login'),
        ),
      ),
    );
  }
}

class HomeScreen extends StatelessWidget {
  const HomeScreen({super.key});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: const Text('Home'),
      ),
      body: Center(
        child: Column(
          mainAxisAlignment: MainAxisAlignment.center,
          children: [
            const Text('Welcome!'),
            const SizedBox(height: 20),
            ElevatedButton(
              onPressed: () {
                // Push to ProfileScreen normally
                // Navigation stack: [/home, /profile]
                Navigator.push(
                  context,
                  MaterialPageRoute(
                    builder: (context) => const ProfileScreen(),
                  ),
                );
              },
              child: const Text('Go to Profile'),
            ),
            const SizedBox(height: 20),
            ElevatedButton(
              onPressed: () {
                // pushReplacementNamed is a convenience method for pushReplacement with named routes
                // It uses the routes map defined in MaterialApp
                Navigator.pushReplacementNamed(context, '/profile');
              },
              child: const Text('Replace with Profile'),
            ),
            const SizedBox(height: 20),
            ElevatedButton(
              onPressed: () {
                // pushReplacementAndRemoveUntil removes all previous routes until a condition is met
                // This is useful for clearing the entire navigation stack
                // After this, the navigation stack will be: [/home]
                // All routes before /home are removed
                
                Navigator.pushReplacementAndRemoveUntil(
                  context,
                  MaterialPageRoute(
                    builder: (context) => const LoginScreen(),
                  ),
                  (route) => false,
                  // The predicate returns false for all routes, so all are removed
                  // Only the new route remains
                );
              },
              child: const Text('Clear Stack and Login'),
            ),
          ],
        ),
      ),
    );
  }
}

class ProfileScreen extends StatelessWidget {
  const ProfileScreen({super.key});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: const Text('Profile'),
      ),
      body: Center(
        child: ElevatedButton(
          onPressed: () {
            // Pop normally returns to HomeScreen
            Navigator.pop(context);
          },
          child: const Text('Go Back'),
        ),
      ),
    );
  }
}
```

**Explanation:**

- **`pushReplacement()`**: Replaces the current route with a new route. The current route is removed from the navigation stack, and the new route takes its place. This means the user cannot navigate back to the replaced route using the back button.
- **Use case**: `pushReplacement` is commonly used after a login, registration, or onboarding flow. After the user completes these processes, you don't want them to go back to the login screen.
- **Navigation stack after replacement**: Starting with `[Login]`, after `pushReplacement` to `Home`: `[Home]`. The `Login` route is completely removed from memory.
- **`pushReplacementNamed()`**: A convenience method that combines `pushReplacement` with named routes. It looks up the route builder in the `MaterialApp.routes` map and uses that builder to create the route.
- **`RouteSettings`**: The `settings` parameter allows you to attach metadata to a route, such as a name or custom data. This can be useful for:
  - Analytics and tracking
  - Custom route matching logic
  - Passing additional context to the route
- **`pushReplacementAndRemoveUntil()`**: A more powerful variant that replaces the current route AND removes all previous routes until a predicate returns `true`. If the predicate always returns `false`, all previous routes are removed.
- **Predicate function**: The `(route) => false` predicate is applied to each route in the stack from bottom to top. When it returns `true`, that route and all routes below it are kept. When it returns `false`, that route is removed. Returning `false` for all routes means the entire stack is cleared before adding the new route.
- **Alternative `pushAndRemoveUntil()`**: Similar to `pushReplacementAndRemoveUntil`, but pushes the new route instead of replacing the current one. This can be useful when you want to keep the current route in the stack but clear everything below it.

### **PopUntil and Navigation Stack Management**

```dart
import 'package:flutter/material.dart';

void main() {
  runApp(const PopUntilApp());
}

class PopUntilApp extends StatelessWidget {
  const PopUntilApp({super.key});

  @override
  Widget build(BuildContext context) {
    return MaterialApp(
      title: 'Pop Until',
      home: const RootScreen(),
    );
  }
}

class RootScreen extends StatelessWidget {
  const RootScreen({super.key});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: const Text('Root Screen'),
        automaticallyImplyLeading: false,
      ),
      body: Center(
        child: Column(
          mainAxisAlignment: MainAxisAlignment.center,
          children: [
            ElevatedButton(
              onPressed: () {
                Navigator.push(
                  context,
                  MaterialPageRoute(
                    // RouteSettings with a name allows us to identify routes by name
                    settings: const RouteSettings(name: 'screen1'),
                    builder: (context) => const Screen1(),
                  ),
                );
              },
              child: const Text('Start Navigation Chain'),
            ),
          ],
        ),
      ),
    );
  }
}

class Screen1 extends StatelessWidget {
  const Screen1({super.key});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: const Text('Screen 1'),
      ),
      body: Center(
        child: ElevatedButton(
          onPressed: () {
            Navigator.push(
              context,
              MaterialPageRoute(
                settings: const RouteSettings(name: 'screen2'),
                builder: (context) => const Screen2(),
              ),
            );
          },
          child: const Text('Go to Screen 2'),
        ),
      ),
    );
  }
}

class Screen2 extends StatelessWidget {
  const Screen2({super.key});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: const Text('Screen 2'),
      ),
      body: Center(
        child: ElevatedButton(
          onPressed: () {
            Navigator.push(
              context,
              MaterialPageRoute(
                settings: const RouteSettings(name: 'screen3'),
                builder: (context) => const Screen3(),
              ),
            );
          },
          child: const Text('Go to Screen 3'),
        ),
      ),
    );
  }
}

class Screen3 extends StatelessWidget {
  const Screen3({super.key});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: const Text('Screen 3'),
      ),
      body: Center(
        child: Column(
          mainAxisAlignment: MainAxisAlignment.center,
          children: [
            const Text('Navigation Stack: [Root, 1, 2, 3]'),
            const SizedBox(height: 20),
            ElevatedButton(
              onPressed: () {
                // Pop once: returns to Screen 2
                // Stack becomes: [Root, 1, 2]
                Navigator.pop(context);
              },
              child: const Text('Pop Once (to Screen 2)'),
            ),
            const SizedBox(height: 10),
            ElevatedButton(
              onPressed: () {
                // Pop until reaching a route with a specific name
                // This will pop Screen3 and Screen2, stopping at Screen1
                // Stack becomes: [Root, 1]
                
                Navigator.popUntil(context, (route) {
                  // The predicate is called for each route, starting from the top
                  // When it returns true, popping stops
                  // route.settings.name contains the name we set in RouteSettings
                  
                  print('Checking route: ${route.settings.name}');
                  return route.settings.name == 'screen1';
                });
              },
              child: const Text('Pop Until Screen 1'),
            ),
            const SizedBox(height: 10),
            ElevatedButton(
              onPressed: () {
                // Pop until the route stack has only one route
                // This is a common pattern to return to the root
                // The predicate (route) => route.isFirst checks if this is the first route
                // Stack becomes: [Root]
                
                Navigator.popUntil(context, (route) => route.isFirst);
                // route.isFirst is true for the bottom-most route in the stack
              },
              child: const Text('Pop to Root'),
            ),
            const SizedBox(height: 10),
            ElevatedButton(
              onPressed: () {
                // Pop two routes at once
                // This is equivalent to calling pop() twice, but more efficient
                // Stack becomes: [Root, 1]
                
                Navigator.pop(context); // Pop to Screen 2
                Navigator.pop(context); // Pop to Screen 1
                // Note: These two calls happen sequentially, but quickly
              },
              child: const Text('Pop Two Times'),
            ),
            const SizedBox(height: 10),
            ElevatedButton(
              onPressed: () {
                // popUntil with a custom predicate
                // Pop until we've gone back 2 routes
                int popCount = 0;
                
                Navigator.popUntil(context, (route) {
                  popCount++;
                  // Return true after popping 2 routes
                  // This is a custom way to pop a specific number of routes
                  return popCount == 2;
                });
                // The predicate is evaluated BEFORE popping
                // So popCount reaches 2 when we're at Screen1, and we stop there
              },
              child: const Text('Pop 2 Routes (Custom)'),
            ),
          ],
        ),
      ),
    );
  }
}
```

**Explanation:**

- **`Navigator.popUntil()`**: Pops routes from the stack until a predicate returns `true`. The predicate is a function that takes a `Route` object and returns a `bool`. The predicate is called for each route, starting from the top of the stack.
- **Predicate evaluation**: The predicate is called for each route BEFORE that route is popped. When the predicate returns `true`, that route and all routes below it remain in the stack. The routes above it (that have already been checked) are popped.
- **`route.settings.name`**: The `name` property of `RouteSettings` allows you to identify routes by a string name. This is useful for `popUntil` when you want to pop until you reach a specific named route. You must set the name when creating the route using `RouteSettings(name: 'myRoute')`.
- **`route.isFirst`**: A boolean property that is `true` for the bottom-most route in the stack (the first route pushed). This is commonly used to pop to the root screen.
- **Pop two routes**: While you can call `pop()` twice in succession, this is less efficient than `popUntil` and creates a visual flicker as two transitions happen. `popUntil` performs the operation in one smooth transition.
- **Custom pop count**: The example shows how to pop a specific number of routes using a counter in the predicate. The counter increments each time the predicate is called (for each route checked). When it reaches 2, the predicate returns `true`, stopping at the 2nd route from the top.
- **Common patterns**:
  - `Navigator.popUntil(context, (route) => route.isFirst)`: Pop to root
  - `Navigator.popUntil(context, ModalRoute.withName('/home'))`: Pop to named route (using `ModalRoute.withName`)
  - `Navigator.popUntil(context, (route) => route.settings.name == 'specificRoute')`: Pop to custom named route

---

## **16.3 Passing Data Between Screens**

Passing data between screens is a fundamental requirement for most apps. Flutter provides multiple approaches for passing data, each with different use cases.

### **Passing Data via Constructor (Forward Direction)**

The simplest way to pass data is through the constructor of the destination widget.

```dart
import 'package:flutter/material.dart';

void main() {
  runApp(const DataPassingApp());
}

class DataPassingApp extends StatelessWidget {
  const DataPassingApp({super.key});

  @override
  Widget build(BuildContext context) {
    return MaterialApp(
      title: 'Data Passing',
      home: const UserListScreen(),
    );
  }
}

// Data model class
class User {
  final String id;
  final String name;
  final String email;
  final int age;
  
  User({
    required this.id,
    required this.name,
    required this.email,
    required this.age,
  });
  
  // Factory constructor to create User from JSON
  factory User.fromJson(Map<String, dynamic> json) {
    return User(
      id: json['id'],
      name: json['name'],
      email: json['email'],
      age: json['age'],
    );
  }
  
  // Method to convert User to JSON
  Map<String, dynamic> toJson() {
    return {
      'id': id,
      'name': name,
      'email': email,
      'age': age,
    };
  }
}

class UserListScreen extends StatelessWidget {
  // Sample data list
  final List<User> users = [
    User(id: '1', name: 'Alice Johnson', email: 'alice@example.com', age: 28),
    User(id: '2', name: 'Bob Smith', email: 'bob@example.com', age: 32),
    User(id: '3', name: 'Carol White', email: 'carol@example.com', age: 25),
    User(id: '4', name: 'David Brown', email: 'david@example.com', age: 41),
  ];

  const UserListScreen({super.key});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: const Text('Users'),
      ),
      body: ListView.builder(
        itemCount: users.length,
        itemBuilder: (context, index) {
          final user = users[index];
          return ListTile(
            leading: CircleAvatar(
              // Use first letter of name as avatar
              child: Text(user.name[0]),
            ),
            title: Text(user.name),
            subtitle: Text(user.email),
            trailing: Text('${user.age} years old'),
            onTap: () {
              // Navigate to UserDetailScreen, passing the User object
              // The User object is passed through the constructor
              // This is the simplest and most common way to pass data
              
              Navigator.push(
                context,
                MaterialPageRoute(
                  builder: (context) => UserDetailScreen(user: user),
                  // user: user passes the User object to the constructor
                ),
              );
            },
          );
        },
      ),
    );
  }
}

class UserDetailScreen extends StatelessWidget {
  // The data is received via constructor parameters
  // These parameters are stored as final fields in the widget
  final User user;
  
  // Constructor that requires the User object
  // The 'required' keyword makes this parameter mandatory
  const UserDetailScreen({super.key, required this.user});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: Text(user.name),
      ),
      body: Padding(
        padding: const EdgeInsets.all(16.0),
        child: Column(
          crossAxisAlignment: CrossAxisAlignment.start,
          children: [
            // Display user information
            _buildDetailRow('ID', user.id),
            _buildDetailRow('Name', user.name),
            _buildDetailRow('Email', user.email),
            _buildDetailRow('Age', '${user.age} years'),
            
            const SizedBox(height: 20),
            
            // Show JSON representation
            const Text(
              'JSON Data:',
              style: TextStyle(fontSize: 18, fontWeight: FontWeight.bold),
            ),
            const SizedBox(height: 8),
            Container(
              padding: const EdgeInsets.all(12),
              decoration: BoxDecoration(
                color: Colors.grey[200],
                borderRadius: BorderRadius.circular(8),
              ),
              child: Text(
                // Convert User to JSON for display
                user.toJson().toString(),
                style: const TextStyle(fontFamily: 'monospace'),
              ),
            ),
            
            const SizedBox(height: 20),
            
            // Button to edit user
            ElevatedButton(
              onPressed: () {
                // Navigate to EditUserScreen, passing the same User object
                // We can pass the same object to multiple screens
                Navigator.push(
                  context,
                  MaterialPageRoute(
                    builder: (context) => EditUserScreen(user: user),
                  ),
                );
              },
              child: const Text('Edit User'),
            ),
          ],
        ),
      ),
    );
  }
  
  // Helper method to build a detail row
  Widget _buildDetailRow(String label, String value) {
    return Padding(
      padding: const EdgeInsets.only(bottom: 12),
      child: Row(
        crossAxisAlignment: CrossAxisAlignment.start,
        children: [
          SizedBox(
            width: 80,
            child: Text(
              '$label:',
              style: const TextStyle(fontWeight: FontWeight.bold),
            ),
          ),
          Expanded(
            child: Text(value),
          ),
        ],
      ),
    );
  }
}

class EditUserScreen extends StatefulWidget {
  final User user;
  
  const EditUserScreen({super.key, required this.user});

  @override
  State<EditUserScreen> createState() => _EditUserScreenState();
}

class _EditUserScreenState extends State<EditUserScreen> {
  // Controllers for text fields
  late final TextEditingController _nameController;
  late final TextEditingController _emailController;
  late final TextEditingController _ageController;
  
  @override
  void initState() {
    super.initState();
    // Initialize controllers with user data
    // widget.user gives access to the user passed via constructor
    _nameController = TextEditingController(text: widget.user.name);
    _emailController = TextEditingController(text: widget.user.email);
    _ageController = TextEditingController(text: widget.user.age.toString());
  }
  
  @override
  void dispose() {
    // Always dispose controllers to prevent memory leaks
    _nameController.dispose();
    _emailController.dispose();
    _ageController.dispose();
    super.dispose();
  }

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: const Text('Edit User'),
        actions: [
          IconButton(
            icon: const Icon(Icons.save),
            onPressed: () {
              // Create a new User object with updated data
              final updatedUser = User(
                id: widget.user.id, // ID doesn't change
                name: _nameController.text,
                email: _emailController.text,
                age: int.tryParse(_ageController.text) ?? widget.user.age,
              );
              
              // Pop with the updated user as the result
              // This passes data back to the previous screen
              Navigator.pop(context, updatedUser);
            },
          ),
        ],
      ),
      body: Padding(
        padding: const EdgeInsets.all(16.0),
        child: Column(
          children: [
            TextField(
              controller: _nameController,
              decoration: const InputDecoration(
                labelText: 'Name',
                border: OutlineInputBorder(),
              ),
            ),
            const SizedBox(height: 16),
            TextField(
              controller: _emailController,
              decoration: const InputDecoration(
                labelText: 'Email',
                border: OutlineInputBorder(),
              ),
              keyboardType: TextInputType.emailAddress,
            ),
            const SizedBox(height: 16),
            TextField(
              controller: _ageController,
              decoration: const InputDecoration(
                labelText: 'Age',
                border: OutlineInputBorder(),
              ),
              keyboardType: TextInputType.number,
            ),
          ],
        ),
      ),
    );
  }
}
```

**Explanation:**

- **Constructor parameters**: Data is passed by defining parameters in the widget's constructor and storing them as `final` fields. This is the simplest and most type-safe way to pass data. The compiler ensures that all required data is provided.
- **`required` keyword**: The `required` keyword makes a constructor parameter mandatory. If you try to create the widget without providing this parameter, the compiler will show an error. This helps prevent runtime errors caused by missing data.
- **Data model class**: The `User` class is a data model that encapsulates related data. Using a data model (instead of passing multiple individual parameters) keeps the code organized and makes it easier to pass data between screens.
- **`fromJson` and `toJson`**: These factory and instance methods allow conversion between the `User` object and JSON (a Map). This is essential when data comes from an API or needs to be stored/sent over the network.
- **Accessing passed data**: In the destination widget, use `widget.parameterName` to access the data passed via the constructor. For example, `widget.user.name` accesses the `name` field of the `user` object.
- **StatefulWidget with passed data**: When passing data to a `StatefulWidget`, the data is passed to the widget class (which is immutable) and accessed via `widget.dataName` in the state class.
- **`initState`**: Use `initState` to initialize controllers or perform one-time setup that depends on the passed data. `initState` is called once when the state object is first created.
- **`dispose`**: Always dispose of controllers (`TextEditingController`, `AnimationController`, etc.) in the `dispose` method to prevent memory leaks. This is called when the widget is removed from the widget tree permanently.
- **Returning updated data**: After editing data, you can pass it back to the previous screen using `Navigator.pop(context, updatedData)`. The previous screen can capture this data by awaiting the `Future` returned from its `push` call.

### **Receiving Data from Previous Screen (Backward Direction)**

```dart
import 'package:flutter/material.dart';

void main() {
  runApp(const ReturnDataApp());
}

class ReturnDataApp extends StatelessWidget {
  const ReturnDataApp({super.key});

  @override
  Widget build(BuildContext context) {
    return MaterialApp(
      title: 'Return Data',
      home: const ProductListScreen(),
    );
  }
}

// Data model for a product
class Product {
  final String id;
  final String name;
  final double price;
  final String imageUrl;
  
  Product({
    required this.id,
    required this.name,
    required this.price,
    required this.imageUrl,
  });
}

// Data model for a cart item
class CartItem {
  final Product product;
  int quantity;
  
  CartItem({required this.product, this.quantity = 1});
  
  double get total => product.price * quantity;
}

class ProductListScreen extends StatefulWidget {
  const ProductListScreen({super.key});

  @override
  State<ProductListScreen> createState() => _ProductListScreenState();
}

class _ProductListScreenState extends State<ProductListScreen> {
  // List of products
  final List<Product> products = [
    Product(
      id: '1',
      name: 'Laptop',
      price: 999.99,
      imageUrl: 'https://via.placeholder.com/150',
    ),
    Product(
      id: '2',
      name: 'Smartphone',
      price: 699.99,
      imageUrl: 'https://via.placeholder.com/150',
    ),
    Product(
      id: '3',
      name: 'Headphones',
      price: 149.99,
      imageUrl: 'https://via.placeholder.com/150',
    ),
  ];
  
  // Cart to store selected items
  final List<CartItem> cart = [];

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: const Text('Products'),
        actions: [
          // Cart icon with badge
          Stack(
            children: [
              const IconButton(
                icon: Icon(Icons.shopping_cart),
                onPressed: null, // Navigate to cart screen
              ),
              if (cart.isNotEmpty)
                Positioned(
                  right: 0,
                  top: 0,
                  child: Container(
                    padding: const EdgeInsets.all(2),
                    decoration: BoxDecoration(
                      color: Colors.red,
                      borderRadius: BorderRadius.circular(10),
                    ),
                    constraints: const BoxConstraints(
                      minWidth: 16,
                      minHeight: 16,
                    ),
                    child: Text(
                      '${cart.length}',
                      style: const TextStyle(
                        color: Colors.white,
                        fontSize: 10,
                      ),
                      textAlign: TextAlign.center,
                    ),
                  ),
                ),
          ],
        ],
      ),
      body: ListView.builder(
        itemCount: products.length,
        itemBuilder: (context, index) {
          final product = products[index];
          return Card(
            margin: const EdgeInsets.all(8),
            child: ListTile(
              leading: Image.network(
                product.imageUrl,
                width: 50,
                height: 50,
                fit: BoxFit.cover,
              ),
              title: Text(product.name),
              subtitle: Text('\$${product.price.toStringAsFixed(2)}'),
              trailing: ElevatedButton(
                onPressed: () async {
                  // Navigate to ProductDetailScreen and await the result
                  // The result is a CartItem if the user added to cart
                  // or null if they didn't
                  
                  final result = await Navigator.push<CartItem>(
                    context,
                    MaterialPageRoute(
                      builder: (context) => ProductDetailScreen(product: product),
                    ),
                  );
                  
                  // If the user added the product to cart, add it to our cart list
                  if (result != null) {
                    setState(() {
                      cart.add(result);
                      // Show a snackbar to confirm the addition
                      ScaffoldMessenger.of(context).showSnackBar(
                        SnackBar(
                          content: Text('${result.product.name} added to cart'),
                          duration: const Duration(seconds: 2),
                        ),
                      );
                    });
                  }
                },
                child: const Text('View'),
              ),
            ),
          );
        },
      ),
    );
  }
}

class ProductDetailScreen extends StatelessWidget {
  final Product product;
  
  const ProductDetailScreen({super.key, required this.product});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: Text(product.name),
      ),
      body: SingleChildScrollView(
        child: Column(
          crossAxisAlignment: CrossAxisAlignment.stretch,
          children: [
            // Product image
            Image.network(
              product.imageUrl,
              height: 300,
              fit: BoxFit.cover,
            ),
            
            Padding(
              padding: const EdgeInsets.all(16),
              child: Column(
                crossAxisAlignment: CrossAxisAlignment.start,
                children: [
                  // Product name
                  Text(
                    product.name,
                    style: const TextStyle(
                      fontSize: 24,
                      fontWeight: FontWeight.bold,
                    ),
                  ),
                  
                  const SizedBox(height: 8),
                  
                  // Product price
                  Text(
                    '\$${product.price.toStringAsFixed(2)}',
                    style: const TextStyle(
                      fontSize: 20,
                      color: Colors.green,
                      fontWeight: FontWeight.bold,
                    ),
                  ),
                  
                  const SizedBox(height: 16),
                  
                  // Product description (placeholder)
                  const Text(
                    'This is a high-quality product with excellent features. '
                    'Made with premium materials and designed for durability.',
                    style: TextStyle(fontSize: 16),
                  ),
                  
                  const SizedBox(height: 24),
                  
                  // Quantity selector
                  QuantitySelector(
                    // Callback when quantity is selected
                    // This will navigate back with the CartItem
                    onAddToCart: (quantity) {
                      // Create a CartItem with the selected quantity
                      final cartItem = CartItem(
                        product: product,
                        quantity: quantity,
                      );
                      
                      // Pop and return the CartItem as the result
                      // This completes the Future in ProductListScreen
                      Navigator.pop(context, cartItem);
                    },
                  ),
                ],
              ),
            ),
          ],
        ),
      ),
    );
  }
}

class QuantitySelector extends StatefulWidget {
  final Function(int) onAddToCart;
  
  const QuantitySelector({super.key, required this.onAddToCart});

  @override
  State<QuantitySelector> createState() => _QuantitySelectorState();
}

class _QuantitySelectorState extends State<QuantitySelector> {
  int _quantity = 1;

  @override
  Widget build(BuildContext context) {
    return Row(
      children: [
        // Decrease button
        IconButton(
          icon: const Icon(Icons.remove),
          onPressed: _quantity > 1
              ? () {
                  setState(() {
                    _quantity--;
                  });
                }
              : null, // Disable when quantity is 1
        ),
        
        // Quantity display
        Container(
          padding: const EdgeInsets.symmetric(horizontal: 16, vertical: 8),
          decoration: BoxDecoration(
            border: Border.all(color: Colors.grey),
            borderRadius: BorderRadius.circular(4),
          ),
          child: Text(
            '$_quantity',
            style: const TextStyle(fontSize: 18),
          ),
        ),
        
        // Increase button
        IconButton(
          icon: const Icon(Icons.add),
          onPressed: () {
            setState(() {
              _quantity++;
            });
          },
        ),
        
        const Spacer(),
        
        // Add to cart button
        ElevatedButton.icon(
          onPressed: () {
            // Call the callback with the selected quantity
            // This will trigger Navigator.pop in ProductDetailScreen
            widget.onAddToCart(_quantity);
          },
          icon: const Icon(Icons.shopping_cart),
          label: const Text('Add to Cart'),
          style: ElevatedButton.styleFrom(
            padding: const EdgeInsets.symmetric(horizontal: 24, vertical: 12),
          ),
        ),
      ],
    );
  }
}
```

**Explanation:**

- **`await Navigator.push<CartItem>()`**: The generic type parameter `<CartItem>` specifies the type of data expected to be returned. This provides type safety - the `result` will be of type `CartItem?` (nullable), and the IDE can provide better autocomplete and type checking.
- **Returning data with `Navigator.pop()`**: When the user adds an item to the cart, we call `Navigator.pop(context, cartItem)`. This passes the `cartItem` back to the previous screen. The `Future` returned by `push` completes with this value.
- **Null result**: If the user navigates back using the system back button (or another `pop` without a result), the `result` will be `null`. Always check `if (result != null)` before using the result to avoid null pointer errors.
- **`setState`**: When the result is received and added to the cart, we call `setState` to rebuild the UI. This updates the cart badge and shows a snackbar confirmation.
- **Callback pattern**: The `QuantitySelector` widget uses a callback (`onAddToCart`) to communicate the selected quantity back to its parent. This is a common pattern for child-to-parent communication within the same screen.
- **Snackbar feedback**: `ScaffoldMessenger.of(context).showSnackBar()` displays a temporary message to confirm the action. This provides immediate feedback to the user.
- **Type safety**: By using `await Navigator.push<CartItem>()`, the compiler knows that `result` is of type `CartItem?`. This prevents type errors and makes the code more maintainable.

### **Passing Complex Objects and Multiple Parameters**

```dart
import 'package:flutter/material.dart';

void main() {
  runApp(const ComplexDataApp());
}

class ComplexDataApp extends StatelessWidget {
  const ComplexDataApp({super.key});

  @override
  Widget build(BuildContext context) {
    return MaterialApp(
      title: 'Complex Data',
      home: const BookingScreen(),
    );
  }
}

// Complex data model for a booking
class BookingRequest {
  final String destination;
  final DateTime departureDate;
  final DateTime returnDate;
  final int numberOfTravelers;
  final TravelClass travelClass;
  final List<String> selectedAmenities;
  final bool travelInsurance;
  
  BookingRequest({
    required this.destination,
    required this.departureDate,
    required this.returnDate,
    required this.numberOfTravelers,
    required this.travelClass,
    required this.selectedAmenities,
    required this.travelInsurance,
  });
  
  // Calculate total price based on booking details
  double calculateTotalPrice() {
    double basePrice = switch (travelClass) {
      TravelClass.economy => 500,
      TravelClass.business => 1500,
      TravelClass.firstClass => 3000,
    };
    
    double totalPrice = basePrice * numberOfTravelers;
    
    // Add amenity costs
    totalPrice += selectedAmenities.length * 50;
    
    // Add insurance cost
    if (travelInsurance) {
      totalPrice += 100 * numberOfTravelers;
    }
    
    return totalPrice;
  }
}

enum TravelClass { economy, business, firstClass }

class BookingScreen extends StatelessWidget {
  const BookingScreen({super.key});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: const Text('Book a Trip'),
      ),
      body: Center(
        child: ElevatedButton(
          onPressed: () async {
            // Navigate to the multi-step booking wizard
            // The wizard returns a complete BookingRequest when finished
            
            final bookingRequest = await Navigator.push<BookingRequest>(
              context,
              MaterialPageRoute(
                builder: (context) => const BookingWizard(),
              ),
            );
            
            // Show booking confirmation if a request was returned
            if (bookingRequest != null) {
              _showBookingConfirmation(context, bookingRequest);
            }
          },
          child: const Text('Start Booking'),
        ),
      ),
    );
  }
  
  void _showBookingConfirmation(BuildContext context, BookingRequest request) {
    showDialog(
      context: context,
      builder: (context) => AlertDialog(
        title: const Text('Booking Confirmed!'),
        content: Column(
          mainAxisSize: MainAxisSize.min,
          crossAxisAlignment: CrossAxisAlignment.start,
          children: [
            Text('Destination: ${request.destination}'),
            Text('Departure: ${_formatDate(request.departureDate)}'),
            Text('Return: ${_formatDate(request.returnDate)}'),
            Text('Travelers: ${request.numberOfTravelers}'),
            Text('Class: ${request.travelClass.name}'),
            Text('Amenities: ${request.selectedAmenities.join(", ")}'),
            Text('Insurance: ${request.travelInsurance ? "Yes" : "No"}'),
            const SizedBox(height: 16),
            Text(
              'Total: \$${request.calculateTotalPrice().toStringAsFixed(2)}',
              style: const TextStyle(
                fontSize: 20,
                fontWeight: FontWeight.bold,
                color: Colors.green,
              ),
            ),
          ],
        ),
        actions: [
          TextButton(
            onPressed: () => Navigator.pop(context),
            child: const Text('OK'),
          ),
        ],
      ),
    );
  }
  
  String _formatDate(DateTime date) {
    return '${date.day}/${date.month}/${date.year}';
  }
}

// Multi-step booking wizard
class BookingWizard extends StatefulWidget {
  const BookingWizard({super.key});

  @override
  State<BookingWizard> createState() => _BookingWizardState();
}

class _BookingWizardState extends State<BookingWizard> {
  // Current step in the wizard
  int _currentStep = 0;
  
  // Accumulate booking data as user progresses
  // Each step stores partial data
  String? _destination;
  DateTime? _departureDate;
  DateTime? _returnDate;
  int _numberOfTravelers = 1;
  TravelClass _travelClass = TravelClass.economy;
  final List<String> _selectedAmenities = [];
  bool _travelInsurance = false;
  
  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: const Text('Booking Wizard'),
      ),
      body: Stepper(
        // Stepper widget for multi-step forms
        currentStep: _currentStep,
        onStepContinue: _handleContinue,
        onStepCancel: _handleCancel,
        steps: [
          // Step 1: Destination
          Step(
            title: const Text('Destination'),
            content: TextField(
              decoration: const InputDecoration(
                labelText: 'Where do you want to go?',
                border: OutlineInputBorder(),
              ),
              onChanged: (value) {
                _destination = value;
              },
            ),
            isActive: _currentStep == 0,
          ),
          
          // Step 2: Dates
          Step(
            title: const Text('Dates'),
            content: Column(
              children: [
                ListTile(
                  title: const Text('Departure Date'),
                  subtitle: Text(_departureDate != null
                      ? _formatDate(_departureDate!)
                      : 'Not selected'),
                  trailing: const Icon(Icons.calendar_today),
                  onTap: () => _selectDate(context, true),
                ),
                ListTile(
                  title: const Text('Return Date'),
                  subtitle: Text(_returnDate != null
                      ? _formatDate(_returnDate!)
                      : 'Not selected'),
                  trailing: const Icon(Icons.calendar_today),
                  onTap: () => _selectDate(context, false),
                ),
              ],
            ),
            isActive: _currentStep == 1,
          ),
          
          // Step 3: Travelers
          Step(
            title: const Text('Travelers'),
            content: Row(
              children: [
                IconButton(
                  icon: const Icon(Icons.remove),
                  onPressed: _numberOfTravelers > 1
                      ? () => setState(() => _numberOfTravelers--)
                      : null,
                ),
                Text('$_numberOfTravelers'),
                IconButton(
                  icon: const Icon(Icons.add),
                  onPressed: () => setState(() => _numberOfTravelers++),
                ),
              ],
            ),
            isActive: _currentStep == 2,
          ),
          
          // Step 4: Class & Amenities
          Step(
            title: const Text('Class & Amenities'),
            content: Column(
              children: [
                // Travel class selection
                DropdownButton<TravelClass>(
                  value: _travelClass,
                  items: TravelClass.values.map((travelClass) {
                    return DropdownMenuItem(
                      value: travelClass,
                      child: Text(travelClass.name),
                    );
                  }).toList(),
                  onChanged: (value) {
                    if (value != null) {
                      setState(() => _travelClass = value);
                    }
                  },
                ),
                
                const SizedBox(height: 16),
                
                // Amenities checkboxes
                const Text('Select Amenities:'),
                CheckboxListTile(
                  title: const Text('WiFi'),
                  value: _selectedAmenities.contains('WiFi'),
                  onChanged: (value) {
                    setState(() {
                      if (value == true) {
                        _selectedAmenities.add('WiFi');
                      } else {
                        _selectedAmenities.remove('WiFi');
                      }
                    });
                  },
                ),
                CheckboxListTile(
                  title: const Text('Meals'),
                  value: _selectedAmenities.contains('Meals'),
                  onChanged: (value) {
                    setState(() {
                      if (value == true) {
                        _selectedAmenities.add('Meals');
                      } else {
                        _selectedAmenities.remove('Meals');
                      }
                    });
                  },
                ),
                CheckboxListTile(
                  title: const Text('Extra Legroom'),
                  value: _selectedAmenities.contains('Extra Legroom'),
                  onChanged: (value) {
                    setState(() {
                      if (value == true) {
                        _selectedAmenities.add('Extra Legroom');
                      } else {
                        _selectedAmenities.remove('Extra Legroom');
                      }
                    });
                  },
                ),
              ],
            ),
            isActive: _currentStep == 3,
          ),
          
          // Step 5: Insurance
          Step(
            title: const Text('Insurance'),
            content: SwitchListTile(
              title: const Text('Travel Insurance'),
              subtitle: const Text('Protect your trip with insurance'),
              value: _travelInsurance,
              onChanged: (value) {
                setState(() => _travelInsurance = value);
              },
            ),
            isActive: _currentStep == 4,
          ),
          
          // Step 6: Review
          Step(
            title: const Text('Review'),
            content: _buildReviewContent(),
            isActive: _currentStep == 5,
          ),
        ],
      ),
    );
  }
  
  Widget _buildReviewContent() {
    return Column(
      crossAxisAlignment: CrossAxisAlignment.start,
      children: [
        _buildReviewRow('Destination', _destination ?? 'Not selected'),
        _buildReviewRow('Departure', _formatDate(_departureDate)),
        _buildReviewRow('Return', _formatDate(_returnDate)),
        _buildReviewRow('Travelers', '$_numberOfTravelers'),
        _buildReviewRow('Class', _travelClass.name),
        _buildReviewRow('Amenities', _selectedAmenities.join(', ') ?? 'None'),
        _buildReviewRow('Insurance', _travelInsurance ? 'Yes' : 'No'),
      ],
    );
  }
  
  Widget _buildReviewRow(String label, String value) {
    return Padding(
      padding: const EdgeInsets.symmetric(vertical: 4),
      child: Row(
        children: [
          SizedBox(
            width: 100,
            child: Text(
              '$label:',
              style: const TextStyle(fontWeight: FontWeight.bold),
            ),
          ),
          Expanded(child: Text(value)),
        ],
      ),
    );
  }
  
  void _handleContinue() {
    // Validation before proceeding
    if (_currentStep == 0 && (_destination == null || _destination!.isEmpty)) {
      ScaffoldMessenger.of(context).showSnackBar(
        const SnackBar(content: Text('Please enter a destination')),
      );
      return;
    }
    
    if (_currentStep == 1 && (_departureDate == null || _returnDate == null)) {
      ScaffoldMessenger.of(context).showSnackBar(
        const SnackBar(content: Text('Please select both dates')),
      );
      return;
    }
    
    // Move to next step or finish
    if (_currentStep < 5) {
      setState(() {
        _currentStep++;
      });
    } else {
      // All steps completed, create BookingRequest
      final bookingRequest = BookingRequest(
        destination: _destination!,
        departureDate: _departureDate!,
        returnDate: _returnDate!,
        numberOfTravelers: _numberOfTravelers,
        travelClass: _travelClass,
        selectedAmenities: _selectedAmenities,
        travelInsurance: _travelInsurance,
      );
      
      // Return the complete booking request
      Navigator.pop(context, bookingRequest);
    }
  }
  
  void _handleCancel() {
    if (_currentStep > 0) {
      setState(() {
        _currentStep--;
      });
    } else {
      // Cancel the entire wizard
      Navigator.pop(context);
    }
  }
  
  Future<void> _selectDate(BuildContext context, bool isDeparture) async {
    final selectedDate = await showDatePicker(
      context: context,
      initialDate: DateTime.now(),
      firstDate: DateTime.now(),
      lastDate: DateTime.now().add(const Duration(days: 365)),
    );
    
    if (selectedDate != null) {
      setState(() {
        if (isDeparture) {
          _departureDate = selectedDate;
        } else {
          _returnDate = selectedDate;
        }
      });
    }
  }
  
  String _formatDate(DateTime date) {
    return '${date.day}/${date.month}/${date.year}';
  }
}
```

**Explanation:**

- **Complex data model**: The `BookingRequest` class encapsulates all the data for a booking, including multiple fields of different types (strings, dates, enums, lists, booleans). This keeps related data together and makes it easy to pass between screens.
- **Enum usage**: The `TravelClass` enum defines a fixed set of options for the travel class. Using enums provides type safety and makes the code more readable than using strings or integers.
- **Calculated properties**: The `calculateTotalPrice()` method computes the total price based on the booking details. This encapsulates business logic within the data model, keeping the UI code clean.
- **Multi-step form (Stepper)**: The `Stepper` widget is ideal for breaking complex forms into manageable steps. Each step collects a subset of data, and the final step reviews all the data before submission.
- **Accumulating data**: The wizard state holds partial data as the user progresses through the steps. Each step updates specific fields, and the final step creates the complete `BookingRequest` object.
- **Validation**: Before moving to the next step, the code validates that required fields are filled. If validation fails, a snackbar is shown to inform the user.
- **Date picker**: The `showDatePicker` function displays a platform-specific date picker. It returns a `Future<DateTime?>` that completes when the user selects a date or cancels.
- **DropdownButton**: Used for selecting the travel class from the enum values. This provides a user-friendly way to select from a fixed set of options.
- **CheckboxListTile**: Used for selecting multiple amenities. Each checkbox independently adds or removes the amenity from the `_selectedAmenities` list.
- **SwitchListTile**: Used for the binary insurance choice. It's more touch-friendly than a standard checkbox and clearly indicates an on/off state.
- **Final assembly**: When the user completes all steps, the `BookingRequest` object is created with all the accumulated data and passed back via `Navigator.pop()`.

---

## **16.4 Named Routes and Route Maps**

Named routes provide a centralized way to manage navigation in your app. Instead of defining routes inline, you define them once in a map and navigate using string names.

### **Setting Up Named Routes**

```dart
import 'package:flutter/material.dart';

void main() {
  runApp(const NamedRoutesApp());
}

class NamedRoutesApp extends StatelessWidget {
  const NamedRoutesApp({super.key});

  @override
  Widget build(BuildContext context) {
    return MaterialApp(
      title: 'Named Routes',
      // The routes map defines all named routes in the app
      // Each route name is a string (the path)
      // Each route value is a WidgetBuilder function that returns the screen
      routes: {
        // '/' is the home route (initial route)
        '/': (context) => const HomeScreen(),
        
        // '/profile' navigates to the ProfileScreen
        '/profile': (context) => const ProfileScreen(),
        
        // '/settings' navigates to the SettingsScreen
        '/settings': (context) => const SettingsScreen(),
        
        // '/details' navigates to the DetailsScreen
        '/details': (context) => const DetailsScreen(),
        
        // '/about' navigates to the AboutScreen
        '/about': (context) => const AboutScreen(),
      },
      
      // initialRoute sets the starting route when the app launches
      // If not specified, '/' is used by default
      initialRoute: '/',
      
      // onUnknownRoute is called when a route is not found in the routes map
      // This is useful for showing a 404 error screen
      onUnknownRoute: (settings) {
        return MaterialPageRoute(
          builder: (context) => const NotFoundScreen(),
        );
      },
    );
  }
}

class HomeScreen extends StatelessWidget {
  const HomeScreen({super.key});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: const Text('Home'),
      ),
      body: ListView(
        children: [
          ListTile(
            title: const Text('Go to Profile'),
            onTap: () {
              // Navigate using the named route
              // Navigator.pushNamed looks up the route in the routes map
              Navigator.pushNamed(context, '/profile');
            },
          ),
          ListTile(
            title: const Text('Go to Settings'),
            onTap: () {
              Navigator.pushNamed(context, '/settings');
            },
          ),
          ListTile(
            title: const Text('Go to Details'),
            onTap: () {
              Navigator.pushNamed(context, '/details');
            },
          ),
          ListTile(
            title: const Text('Go to About'),
            onTap: () {
              Navigator.pushNamed(context, '/about');
            },
          ),
          ListTile(
            title: const Text('Go to Unknown Route (404)'),
            onTap: () {
              // This route is not defined, so onUnknownRoute will be called
              Navigator.pushNamed(context, '/nonexistent');
            },
          ),
        ],
      ),
    );
  }
}

class ProfileScreen extends StatelessWidget {
  const ProfileScreen({super.key});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: const Text('Profile'),
      ),
      body: Center(
        child: Column(
          mainAxisAlignment: MainAxisAlignment.center,
          children: [
            const CircleAvatar(
              radius: 50,
              child: Icon(Icons.person, size: 50),
            ),
            const SizedBox(height: 20),
            const Text('John Doe'),
            const SizedBox(height: 20),
            ElevatedButton(
              onPressed: () => Navigator.pop(context),
              child: const Text('Go Back'),
            ),
          ],
        ),
      ),
    );
  }
}

class SettingsScreen extends StatelessWidget {
  const SettingsScreen({super.key});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: const Text('Settings'),
      ),
      body: ListView(
        children: const [
          ListTile(
            title: Text('Notifications'),
            trailing: Switch(value: true, onChanged: null),
          ),
          ListTile(
            title: Text('Dark Mode'),
            trailing: Switch(value: false, onChanged: null),
          ),
          ListTile(
            title: Text('Language'),
            subtitle: Text('English'),
          ),
        ],
      ),
    );
  }
}

class DetailsScreen extends StatelessWidget {
  const DetailsScreen({super.key});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: const Text('Details'),
      ),
      body: const Center(
        child: Text('Details Screen Content'),
      ),
    );
  }
}

class AboutScreen extends StatelessWidget {
  const AboutScreen({super.key});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: const Text('About'),
      ),
      body: const Center(
        child: Text('About Screen Content'),
      ),
    );
  }
}

class NotFoundScreen extends StatelessWidget {
  const NotFoundScreen({super.key});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: const Text('Not Found'),
      ),
      body: Center(
        child: Column(
          mainAxisAlignment: MainAxisAlignment.center,
          children: [
            const Icon(
              Icons.error_outline,
              size: 64,
              color: Colors.red,
            ),
            const SizedBox(height: 16),
            const Text(
              '404 - Page Not Found',
              style: TextStyle(fontSize: 24),
            ),
            const SizedBox(height: 16),
            ElevatedButton(
              onPressed: () => Navigator.pop(context),
              child: const Text('Go Back'),
            ),
          ],
        ),
      ),
    );
  }
}
```

**Explanation:**

- **`routes` map**: The `routes` parameter in `MaterialApp` takes a `Map<String, WidgetBuilder>`. The keys are route names (strings that look like paths), and the values are functions that build the screen widgets.
- **Home route (`'/'`)**: The `'/'` route is conventionally used as the home or initial route. It's the root of your app's navigation hierarchy.
- **`Navigator.pushNamed()`**: This method looks up the route name in the `routes` map and navigates to the corresponding screen. It's equivalent to calling `Navigator.push(context, MaterialPageRoute(builder: (context) => ...))` but uses the pre-defined route from the map.
- **`initialRoute`**: Specifies which route to show when the app first launches. If not set, `'/'` is used by default. This is useful for showing a splash screen, login screen, or onboarding flow on first launch.
- **`onUnknownRoute`**: Called when a route name is not found in the `routes` map. This is essential for handling invalid routes (typos, deep links to non-existent pages, etc.). You typically show a "404 Not Found" screen.
- **Route naming convention**: Use path-like strings (e.g., `'/profile'`, `'/settings'`, `'/product/123'`) for route names. This makes deep linking easier and aligns with web conventions if you later add web support.
- **Centralized route management**: All routes are defined in one place (the `routes` map), making it easy to see all possible screens in your app and ensuring consistency in navigation.

### **Passing Arguments with Named Routes**

Named routes can receive arguments, allowing you to pass data to the destination screen.

```dart
import 'package:flutter/material.dart';

void main() {
  runApp(const ArgumentsApp());
}

class ArgumentsApp extends StatelessWidget {
  const ArgumentsApp({super.key});

  @override
  Widget build(BuildContext context) {
    return MaterialApp(
      title: 'Named Routes with Arguments',
      routes: {
        '/': (context) => const HomeScreen(),
        '/product': (context) {
          // Extract arguments from the current route's settings
          // RouteSettings contains the name and arguments for the route
          final args = ModalRoute.of(context)?.settings.arguments;
          
          // Check if arguments are provided and have the correct type
          if (args is Map<String, dynamic>) {
            // Extract the productId from the arguments map
            final productId = args['productId'] as String?;
            final productName = args['productName'] as String?;
            
            // If productId is provided, show the product screen
            if (productId != null) {
              return ProductScreen(
                productId: productId,
                productName: productName ?? 'Unknown Product',
              );
            }
          }
          
          // If arguments are missing or invalid, show an error
          return const ErrorScreen('Missing or invalid product ID');
        },
        '/user': (context) {
          final args = ModalRoute.of(context)?.settings.arguments;
          
          if (args is User) {
            // Arguments can be any type, not just maps
            return UserScreen(user: args);
          }
          
          return const ErrorScreen('Missing user data');
        },
      },
      initialRoute: '/',
    );
  }
}

// Data model
class User {
  final String id;
  final String name;
  final String email;
  
  User({required this.id, required this.name, required this.email});
}

class HomeScreen extends StatelessWidget {
  const HomeScreen({super.key});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: const Text('Home'),
      ),
      body: ListView(
        children: [
          ListTile(
            title: const Text('Navigate to Product with Map Arguments'),
            subtitle: const Text('Pass productId and productName'),
            onTap: () {
              // Navigate with a map of arguments
              Navigator.pushNamed(
                context,
                '/product',
                // arguments is passed to the route's RouteSettings
                // It can be any type (Map, Object, String, etc.)
                arguments: {
                  'productId': 'prod-123',
                  'productName': 'Laptop',
                },
              );
            },
          ),
          ListTile(
            title: const Text('Navigate to Product with Object Arguments'),
            subtitle: const Text('Pass a Product object'),
            onTap: () {
              final product = Product(
                id: 'prod-456',
                name: 'Smartphone',
                price: 699.99,
              );
              
              // Navigate with an object as arguments
              Navigator.pushNamed(
                context,
                '/product',
                arguments: product,
              );
            },
          ),
          ListTile(
            title: const Text('Navigate to User Screen'),
            subtitle: const Text('Pass a User object'),
            onTap: () {
              final user = User(
                id: 'user-789',
                name: 'Alice Johnson',
                email: 'alice@example.com',
              );
              
              Navigator.pushNamed(
                context,
                '/user',
                arguments: user,
              );
            },
          ),
        ],
      ),
    );
  }
}

// Another data model
class Product {
  final String id;
  final String name;
  final double price;
  
  Product({required this.id, required this.name, required this.price});
}

class ProductScreen extends StatelessWidget {
  final String productId;
  final String productName;
  
  // Constructor receives the extracted arguments
  const ProductScreen({
    super.key,
    required this.productId,
    required this.productName,
  });

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: Text(productName),
      ),
      body: Center(
        child: Column(
          mainAxisAlignment: MainAxisAlignment.center,
          children: [
            Text('Product ID: $productId'),
            Text('Product Name: $productName'),
            const SizedBox(height: 20),
            ElevatedButton(
              onPressed: () => Navigator.pop(context),
              child: const Text('Go Back'),
            ),
          ],
        ),
      ),
    );
  }
}

class UserScreen extends StatelessWidget {
  final User user;
  
  const UserScreen({super.key, required this.user});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: Text(user.name),
      ),
      body: Center(
        child: Column(
          mainAxisAlignment: MainAxisAlignment.center,
          children: [
            Text('User ID: ${user.id}'),
            Text('Name: ${user.name}'),
            Text('Email: ${user.email}'),
            const SizedBox(height: 20),
            ElevatedButton(
              onPressed: () => Navigator.pop(context),
              child: const Text('Go Back'),
            ),
          ],
        ),
      ),
    );
  }
}

class ErrorScreen extends StatelessWidget {
  final String message;
  
  const ErrorScreen(this.message, {super.key});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: const Text('Error'),
      ),
      body: Center(
        child: Column(
          mainAxisAlignment: MainAxisAlignment.center,
          children: [
            const Icon(Icons.error, size: 64, color: Colors.red),
            const SizedBox(height: 16),
            Text(message),
            const SizedBox(height: 20),
            ElevatedButton(
              onPressed: () => Navigator.pop(context),
              child: const Text('Go Back'),
            ),
          ],
        ),
      ),
    );
  }
}
```

**Explanation:**

- **`arguments` parameter**: When calling `Navigator.pushNamed()`, you can pass an `arguments` parameter. This can be any type: a `Map`, a custom object, a primitive type, etc. The arguments are stored in the `RouteSettings` of the pushed route.
- **`ModalRoute.of(context)?.settings.arguments`**: In the destination screen, use `ModalRoute.of(context)?.settings.arguments` to retrieve the arguments passed to the route. `ModalRoute` is the base class for most routes (like `MaterialPageRoute`), and its `settings` property contains the `arguments`.
- **Type checking**: Always check the type of arguments before using them. Use `is` to check if arguments are of the expected type. This prevents runtime errors when arguments are missing or of the wrong type.
- **Map arguments**: Using a `Map<String, dynamic>` is a common pattern for passing multiple named arguments. This is flexible and works well when you have several parameters to pass.
- **Object arguments**: You can also pass custom objects (like `User` or `Product`) directly as arguments. This is more type-safe than maps, as the compiler can verify the object's structure.
- **`?` (null-safe)**: `ModalRoute.of(context)` returns `ModalRoute<dynamic>?`, which can be `null`. The `?.` (null-aware) operator safely accesses `settings.arguments`, returning `null` if `ModalRoute.of(context)` is `null`.
- **Error handling**: If arguments are missing or invalid, show an error screen or handle the situation gracefully. This is important for deep linking, where users might navigate directly to a screen without providing the expected arguments.

### **Named Route Variants**

Flutter provides several convenience methods for named navigation with different behaviors.

```dart
import 'package:flutter/material.dart';

void main() {
  runApp(const NamedRouteVariantsApp());
}

class NamedRouteVariantsApp extends StatelessWidget {
  const NamedRouteVariantsApp({super.key});

  @override
  Widget build(BuildContext context) {
    return MaterialApp(
      title: 'Named Route Variants',
      routes: {
        '/': (context) => const HomeScreen(),
        '/login': (context) => const LoginScreen(),
        '/home': (context) => const MainScreen(),
        '/profile': (context) => const ProfileScreen(),
      },
      initialRoute: '/',
    );
  }
}

class HomeScreen extends StatelessWidget {
  const HomeScreen({super.key});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: const Text('Home'),
      ),
      body: ListView(
        children: [
          ListTile(
            title: const Text('pushNamed - Normal Navigation'),
            subtitle: const Text('Adds route to stack'),
            onTap: () {
              // pushNamed: Pushes a named route onto the navigator's stack
              // Equivalent to Navigator.push(context, namedRoute)
              // The user can go back to this screen
              
              Navigator.pushNamed(context, '/login');
            },
          ),
          ListTile(
            title: const Text('pushReplacementNamed - Replace Route'),
            subtitle: const Text('Replaces current route'),
            onTap: () {
              // pushReplacementNamed: Replaces the current route with a named route
              // The current route is removed from the stack
              // The user cannot go back to the replaced route
              // Commonly used after login, registration, or onboarding
              
              Navigator.pushReplacementNamed(context, '/home');
            },
          ),
          ListTile(
            title: const Text('pushNamedAndRemoveUntil - Clear Stack'),
            subtitle: const Text('Removes routes until predicate returns true'),
            onTap: () {
              // pushNamedAndRemoveUntil: Pushes a named route and removes previous
              // routes until a predicate returns true
              // This is useful for clearing the entire navigation stack
              // except for the new route and any routes that match the predicate
              
              Navigator.pushNamedAndRemoveUntil(
                context,
                '/home',
                // The predicate is evaluated for each route
                // When it returns true, that route and all below it are kept
                // When it returns false, that route is removed
                (route) => false,
                // (route) => false removes ALL previous routes
                // Only the new route remains in the stack
              );
            },
          ),
          ListTile(
            title: const Text('popUntil - Pop Multiple Routes'),
            subtitle: const Text('Pops routes until predicate returns true'),
            onTap: () async {
              // First, navigate through several screens
              await Navigator.pushNamed(context, '/login');
              await Navigator.pushNamed(context, '/profile');
              
              // Then, pop until reaching the home route
              Navigator.popUntil(
                context,
                // ModalRoute.withName returns a predicate that checks if a route
                // has the specified name
                ModalRoute.withName('/'),
                // This pops all routes until the '/' route is reached
              );
            },
          ),
        ],
      ),
    );
  }
}

class LoginScreen extends StatelessWidget {
  const LoginScreen({super.key});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: const Text('Login'),
      ),
      body: Center(
        child: Column(
          mainAxisAlignment: MainAxisAlignment.center,
          children: [
            const Text('Login Screen'),
            const SizedBox(height: 20),
            ElevatedButton(
              onPressed: () {
                // Use pushReplacementNamed to prevent going back to login
                Navigator.pushReplacementNamed(context, '/home');
              },
              child: const Text('Login'),
            ),
          ],
        ),
      ),
    );
  }
}

class MainScreen extends StatelessWidget {
  const MainScreen({super.key});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: const Text('Main'),
        automaticallyImplyLeading: false, // Remove back button
      ),
      body: Center(
        child: Column(
          mainAxisAlignment: MainAxisAlignment.center,
          children: [
            const Text('Main Screen'),
            const SizedBox(height: 20),
            ElevatedButton(
              onPressed: () {
                Navigator.pushNamed(context, '/profile');
              },
              child: const Text('Go to Profile'),
            ),
            const SizedBox(height: 20),
            ElevatedButton(
              onPressed: () {
                // Use pushReplacementNamed to replace with login
                // This is useful for logging out
                Navigator.pushReplacementNamed(context, '/login');
              },
              child: const Text('Logout'),
            ),
          ],
        ),
      ),
    );
  }
}

class ProfileScreen extends StatelessWidget {
  const ProfileScreen({super.key});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: const Text('Profile'),
      ),
      body: Center(
        child: ElevatedButton(
          onPressed: () => Navigator.pop(context),
          child: const Text('Go Back'),
        ),
      ),
    );
  }
}
```

**Explanation:**

- **`Navigator.pushNamed()`**: The standard method for navigating to a named route. It adds the route to the navigation stack, allowing the user to go back using the back button or a swipe gesture.
- **`Navigator.pushReplacementNamed()`**: Replaces the current route with the named route. The current route is removed from the stack, so the user cannot navigate back to it. This is the named route equivalent of `pushReplacement`.
- **Use case for `pushReplacementNamed`**: After a successful login, you typically use `pushReplacementNamed` to navigate to the home screen. This prevents the user from going back to the login screen using the back button.
- **`Navigator.pushNamedAndRemoveUntil()`**: A powerful method that pushes a named route AND removes routes from the stack based on a predicate. The predicate is a function that takes a `Route` and returns a `bool`. Routes are removed from the top until the predicate returns `true`.
- **`pushNamedAndRemoveUntil` predicate**: When the predicate returns `false` for all routes, all previous routes are removed. When it returns `true`, that route and all routes below it are kept. This is useful for:
  - Clearing the entire stack after logout
  - Resetting to a specific screen after completing a flow
  - Implementing "deep navigation" that replaces intermediate screens
- **`Navigator.popUntil()`**: Pops routes from the stack until a predicate returns `true`. This is useful for:
  - Going back multiple screens at once
  - Returning to a specific screen (e.g., home) from anywhere in the app
- **`ModalRoute.withName()`**: A helper that creates a predicate which returns `true` for routes with a specific name. This is a convenient way to pop until a named route is reached.
- **`automaticallyImplyLeading: false`**: In the `AppBar`, setting `automaticallyImplyLeading` to `false` removes the back button. This is useful for screens where you don't want the user to go back (e.g., after login).

---

## **16.5 onGenerateRoute and Dynamic Routing**

`onGenerateRoute` provides a callback that is called whenever a named route is pushed. This allows for dynamic route generation based on the route name and arguments.

### **Basic onGenerateRoute Usage**

```dart
import 'package:flutter/material.dart';

void main() {
  runApp(const OnGenerateRouteApp());
}

class OnGenerateRouteApp extends StatelessWidget {
  const OnGenerateRouteApp({super.key});

  @override
  Widget build(BuildContext context) {
    return MaterialApp(
      title: 'onGenerateRoute',
      // onGenerateRoute is called whenever a named route is pushed
      // It allows for dynamic route generation based on the route name and arguments
      // If onGenerateRoute returns null, onUnknownRoute is called
      
      onGenerateRoute: (settings) {
        // settings is a RouteSettings object containing:
        // - name: The route name (e.g., '/product/123')
        // - arguments: Any arguments passed to the route
        
        print('Navigating to: ${settings.name}');
        print('Arguments: ${settings.arguments}');
        
        // Parse the route name to determine which screen to show
        // This allows for dynamic routes based on the path
        
        switch (settings.name) {
          case '/':
            // Home route
            return MaterialPageRoute(
              builder: (context) => const HomeScreen(),
              settings: settings, // Pass along the settings
            );
            
          case '/profile':
            // Profile route
            // Extract user ID from arguments if provided
            final userId = settings.arguments as String?;
            return MaterialPageRoute(
              builder: (context) => ProfileScreen(userId: userId),
              settings: settings,
            );
            
          case '/product':
            // Product route
            // Extract product ID from arguments
            final productId = settings.arguments as String?;
            return MaterialPageRoute(
              builder: (context) => ProductScreen(productId: productId),
              settings: settings,
            );
            
          default:
            // Unknown route - return null to trigger onUnknownRoute
            return null;
        }
      },
      
      // onUnknownRoute is called when onGenerateRoute returns null
      // This is your 404 handler
      onUnknownRoute: (settings) {
        return MaterialPageRoute(
          builder: (context) => NotFoundScreen(routeName: settings.name ?? 'Unknown'),
        );
      },
      
      // initialRoute specifies the starting route
      // onGenerateRoute will be called with this route when the app starts
      initialRoute: '/',
    );
  }
}

class HomeScreen extends StatelessWidget {
  const HomeScreen({super.key});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: const Text('Home'),
      ),
      body: ListView(
        children: [
          ListTile(
            title: const Text('Go to Profile (with user ID)'),
            onTap: () {
              Navigator.pushNamed(
                context,
                '/profile',
                arguments: 'user-123',
              );
            },
          ),
          ListTile(
            title: const Text('Go to Product (with product ID)'),
            onTap: () {
              Navigator.pushNamed(
                context,
                '/product',
                arguments: 'prod-456',
              );
            },
          ),
          ListTile(
            title: const Text('Go to Unknown Route'),
            onTap: () {
              Navigator.pushNamed(context, '/nonexistent');
            },
          ),
        ],
      ),
    );
  }
}

class ProfileScreen extends StatelessWidget {
  final String? userId;
  
  const ProfileScreen({super.key, this.userId});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: const Text('Profile'),
      ),
      body: Center(
        child: Column(
          mainAxisAlignment: MainAxisAlignment.center,
          children: [
            Text(userId != null ? 'User ID: $userId' : 'No user ID provided'),
            const SizedBox(height: 20),
            ElevatedButton(
              onPressed: () => Navigator.pop(context),
              child: const Text('Go Back'),
            ),
          ],
        ),
      ),
    );
  }
}

class ProductScreen extends StatelessWidget {
  final String? productId;
  
  const ProductScreen({super.key, this.productId});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: const Text('Product'),
      ),
      body: Center(
        child: Column(
          mainAxisAlignment: MainAxisAlignment.center,
          children: [
            Text(productId != null ? 'Product ID: $productId' : 'No product ID provided'),
            const SizedBox(height: 20),
            ElevatedButton(
              onPressed: () => Navigator.pop(context),
              child: const Text('Go Back'),
            ),
          ],
        ),
      ),
    );
  }
}

class NotFoundScreen extends StatelessWidget {
  final String routeName;
  
  const NotFoundScreen({super.key, required this.routeName});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: const Text('Not Found'),
      ),
      body: Center(
        child: Column(
          mainAxisAlignment: MainAxisAlignment.center,
          children: [
            const Icon(Icons.error_outline, size: 64, color: Colors.red),
            const SizedBox(height: 16),
            Text('Route not found: $routeName'),
            const SizedBox(height: 20),
            ElevatedButton(
              onPressed: () => Navigator.pop(context),
              child: const Text('Go Back'),
            ),
          ],
        ),
      ),
    );
  }
}
```

**Explanation:**

- **`onGenerateRoute` callback**: This function is called whenever a named route is pushed (including the `initialRoute` when the app starts). It receives a `RouteSettings` object containing the route name and arguments.
- **`RouteSettings` object**: Contains two key properties:
  - `name`: The route name (a string like `'/profile'` or `'/product/123'`)
  - `arguments`: Any data passed to the route via `Navigator.pushNamed(context, route, arguments: data)`
- **Return value**: `onGenerateRoute` must return a `Route` object (typically `MaterialPageRoute` or `CupertinoPageRoute`) or `null`. If it returns `null`, `onUnknownRoute` is called.
- **Dynamic route generation**: The `switch` statement checks the route name and generates the appropriate screen. This allows for conditional logic based on the route, arguments, or other factors.
- **Passing `settings` to `MaterialPageRoute`**: By passing `settings: settings` to the route constructor, the route's settings (including name and arguments) are preserved. This is important for retrieving arguments in the destination screen.
- **`print` statements**: The example prints the route name and arguments to the console. This is useful for debugging navigation flow and understanding what data is being passed.
- **`onUnknownRoute` fallback**: When `onGenerateRoute` returns `null` (for unknown routes), `onUnknownRoute` is called. This provides a consistent way to handle 404 errors.
- **`initialRoute`**: When the app starts, `onGenerateRoute` is called with the `initialRoute` value. This ensures that even the initial route goes through the same route generation logic.

### **Parsing Route Parameters from Path**

You can extract parameters from the route path itself (e.g., `/product/123` where `123` is the product ID).

```dart
import 'package:flutter/material.dart';

void main() {
  runApp(const RouteParametersApp());
}

class RouteParametersApp extends StatelessWidget {
  const RouteParametersApp({super.key});

  @override
  Widget build(BuildContext context) {
    return MaterialApp(
      title: 'Route Parameters',
      onGenerateRoute: (settings) {
        print('Route: ${settings.name}');
        
        // Check if the route name matches a pattern
        // We'll use string operations to extract parameters from the path
        
        if (settings.name == null) {
          return _buildRoute('/', const HomeScreen(), settings);
        }
        
        // Parse /product/:id pattern
        if (settings.name!.startsWith('/product/')) {
          // Extract the ID from the path
          // /product/123 -> '123'
          final productId = settings.name!.split('/')[2];
          
          return MaterialPageRoute(
            builder: (context) => ProductScreen(productId: productId),
            settings: settings,
          );
        }
        
        // Parse /user/:id pattern
        if (settings.name!.startsWith('/user/')) {
          final userId = settings.name!.split('/')[2];
          
          return MaterialPageRoute(
            builder: (context) => UserScreen(userId: userId),
            settings: settings,
          );
        }
        
        // Parse /category/:category/product/:id pattern
        // Example: /category/electronics/product/123
        if (settings.name != null && settings.name!.startsWith('/category/')) {
          final parts = settings.name!.split('/');
          // parts = ['', 'category', 'electronics', 'product', '123']
          
          if (parts.length >= 5 && parts[3] == 'product') {
            final category = parts[2]; // 'electronics'
            final productId = parts[4]; // '123'
            
            return MaterialPageRoute(
              builder: (context) => ProductScreen(
                productId: productId,
                category: category,
              ),
              settings: settings,
            );
          }
        }
        
        // Handle known routes
        switch (settings.name) {
          case '/':
            return _buildRoute('/', const HomeScreen(), settings);
          case '/about':
            return _buildRoute('/about', const AboutScreen(), settings);
          case '/profile':
            return _buildRoute('/profile', const ProfileScreen(), settings);
          default:
            return null; // Will trigger onUnknownRoute
        }
      },
      
      onUnknownRoute: (settings) {
        return MaterialPageRoute(
          builder: (context) => NotFoundScreen(routeName: settings.name ?? 'Unknown'),
        );
      },
      
      initialRoute: '/',
    );
  }
  
  // Helper method to build a route
  MaterialPageRoute _buildRoute(String name, Widget screen, RouteSettings settings) {
    return MaterialPageRoute(
      builder: (context) => screen,
      settings: settings,
    );
  }
}

class HomeScreen extends StatelessWidget {
  const HomeScreen({super.key});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: const Text('Home'),
      ),
      body: ListView(
        children: [
          ListTile(
            title: const Text('Product: /product/123'),
            onTap: () {
              // Navigate with ID in the path
              Navigator.pushNamed(context, '/product/123');
            },
          ),
          ListTile(
            title: const Text('Product: /product/456'),
            onTap: () {
              Navigator.pushNamed(context, '/product/456');
            },
          ),
          ListTile(
            title: const Text('User: /user/alice'),
            onTap: () {
              Navigator.pushNamed(context, '/user/alice');
            },
          ),
          ListTile(
            title: const Text('Category Product: /category/electronics/product/789'),
            onTap: () {
              Navigator.pushNamed(context, '/category/electronics/product/789');
            },
          ),
          ListTile(
            title: const Text('About'),
            onTap: () {
              Navigator.pushNamed(context, '/about');
            },
          ),
          ListTile(
            title: const Text('Profile'),
            onTap: () {
              Navigator.pushNamed(context, '/profile');
            },
          ),
          ListTile(
            title: const Text('Unknown Route'),
            onTap: () {
              Navigator.pushNamed(context, '/invalid/path');
            },
          ),
        ],
      ),
    );
  }
}

class ProductScreen extends StatelessWidget {
  final String productId;
  final String? category; // Optional category parameter
  
  const ProductScreen({
    super.key,
    required this.productId,
    this.category,
  });

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: Text('Product $productId'),
      ),
      body: Center(
        child: Column(
          mainAxisAlignment: MainAxisAlignment.center,
          children: [
            if (category != null)
              Text('Category: $category'),
            Text('Product ID: $productId'),
            const SizedBox(height: 20),
            ElevatedButton(
              onPressed: () => Navigator.pop(context),
              child: const Text('Go Back'),
            ),
          ],
        ),
      ),
    );
  }
}

class UserScreen extends StatelessWidget {
  final String userId;
  
  const UserScreen({super.key, required this.userId});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: Text('User: $userId'),
      ),
      body: Center(
        child: ElevatedButton(
          onPressed: () => Navigator.pop(context),
          child: const Text('Go Back'),
        ),
      ),
    );
  }
}

class AboutScreen extends StatelessWidget {
  const AboutScreen({super.key});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: const Text('About'),
      ),
      body: const Center(
        child: Text('About Screen'),
      ),
    );
  }
}

class ProfileScreen extends StatelessWidget {
  const ProfileScreen({super.key});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: const Text('Profile'),
      ),
      body: const Center(
        child: Text('Profile Screen'),
      ),
    );
  }
}

class NotFoundScreen extends StatelessWidget {
  final String routeName;
  
  const NotFoundScreen({super.key, required this.routeName});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: const Text('Not Found'),
      ),
      body: Center(
        child: Column(
          mainAxisAlignment: MainAxisAlignment.center,
          children: [
            const Icon(Icons.error_outline, size: 64, color: Colors.red),
            const SizedBox(height: 16),
            Text('404: $routeName'),
            const SizedBox(height: 20),
            ElevatedButton(
              onPressed: () => Navigator.pop(context),
              child: const Text('Go Back'),
            ),
          ],
        ),
      ),
    );
  }
}
```

**Explanation:**

- **Path-based parameters**: Instead of using `arguments`, you can embed parameters directly in the route path (e.g., `/product/123`). This is similar to REST API URLs and makes routes more readable.
- **`split('/')`**: The `split` method splits the path string by the `/` delimiter, creating a list of path segments. For `/product/123`, `split('/')` returns `['', 'product', '123']`. Note that the first element is an empty string because the path starts with `/`.
- **Accessing segments**: After splitting, you can access segments by index:
  - `parts[0]`: `''` (empty string before the first `/`)
  - `parts[1]`: `'product'` (the first segment)
  - `parts[2]`: `'123'` (the second segment - the ID)
- **Complex paths**: For nested paths like `/category/electronics/product/123`, `split('/')` returns `['', 'category', 'electronics', 'product', '123']`. You can extract multiple parameters:
  - `parts[2]`: `'electronics'` (the category)
  - `parts[4]`: `'123'` (the product ID)
- **`startsWith()`**: Before attempting to split, check if the route starts with a specific pattern using `startsWith()`. This prevents errors when the route doesn't match the expected pattern.
- **Length checking**: Always check `parts.length` before accessing indices to avoid `RangeError`. For `/category/electronics/product/123`, we check `parts.length >= 5` before accessing `parts[4]`.
- **Optional parameters**: The `category` parameter in `ProductScreen` is optional (`String?`). This allows the same screen to handle both `/product/123` (no category) and `/category/electronics/product/123` (with category).
- **Web compatibility**: Path-based parameters are particularly useful for web support, where the URL in the browser address bar shows the current route. This enables deep linking and sharing specific URLs.

### **Advanced onGenerateRoute with Factory Functions**

```dart
import 'package:flutter/material.dart';

void main() {
  runApp(const AdvancedOnGenerateRouteApp());
}

class AdvancedOnGenerateRouteApp extends StatelessWidget {
  const AdvancedOnGenerateRouteApp({super.key});

  @override
  Widget build(BuildContext context) {
    return MaterialApp(
      title: 'Advanced onGenerateRoute',
      onGenerateRoute: _generateRoute,
      // Pass the function reference instead of defining it inline
      // This keeps the code cleaner and more maintainable
      
      onUnknownRoute: (settings) {
        return MaterialPageRoute(
          builder: (context) => NotFoundScreen(routeName: settings.name ?? 'Unknown'),
        );
      },
      
      initialRoute: '/',
    );
  }
  
  // Separate function for route generation
  // This makes the code more organized and testable
  static Route<dynamic> _generateRoute(RouteSettings settings) {
    print('Generating route for: ${settings.name}');
    
    // Route configurations
    // Using a map to store route builders makes it easy to add new routes
    static const Map<String, RouteBuilder> _routeBuilders = {
      '/': HomeBuilder(),
      '/about': AboutBuilder(),
      '/profile': ProfileBuilder(),
    };
    
    // Check if the route is a known static route
    if (_routeBuilders.containsKey(settings.name)) {
      return _routeBuilders[settings.name]!.build(settings);
    }
    
    // Handle dynamic routes
    if (settings.name?.startsWith('/product/') == true) {
      return ProductBuilder().build(settings);
    }
    
    if (settings.name?.startsWith('/user/') == true) {
      return UserBuilder().build(settings);
    }
    
    if (settings.name?.startsWith('/order/') == true) {
      return OrderBuilder().build(settings);
    }
    
    // Unknown route
    return MaterialPageRoute(
      builder: (context) => NotFoundScreen(routeName: settings.name ?? 'Unknown'),
    );
  }
}

// Abstract base class for route builders
// This provides a consistent interface for building routes
abstract class RouteBuilder {
  Route<dynamic> build(RouteSettings settings);
}

// Home route builder
class HomeBuilder extends RouteBuilder {
  @override
  Route<dynamic> build(RouteSettings settings) {
    return MaterialPageRoute(
      builder: (context) => const HomeScreen(),
      settings: settings,
    );
  }
}

// About route builder
class AboutBuilder extends RouteBuilder {
  @override
  Route<dynamic> build(RouteSettings settings) {
    return MaterialPageRoute(
      builder: (context) => const AboutScreen(),
      settings: settings,
    );
  }
}

// Profile route builder
class ProfileBuilder extends RouteBuilder {
  @override
  Route<dynamic> build(RouteSettings settings) {
    // Extract user ID from arguments
    final userId = settings.arguments as String?;
    
    return MaterialPageRoute(
      builder: (context) => ProfileScreen(userId: userId),
      settings: settings,
    );
  }
}

// Product route builder (dynamic)
class ProductBuilder extends RouteBuilder {
  @override
  Route<dynamic> build(RouteSettings settings) {
    // Extract product ID from path
    final productId = settings.name?.split('/')[2];
    
    // Extract category from arguments if provided
    final category = settings.arguments as String?;
    
    if (productId == null) {
      // If no ID is provided, show an error
      return MaterialPageRoute(
        builder: (context) => const ErrorScreen('Missing product ID'),
        settings: settings,
      );
    }
    
    return MaterialPageRoute(
      builder: (context) => ProductScreen(
        productId: productId,
        category: category,
      ),
      settings: settings,
    );
  }
}

// User route builder (dynamic)
class UserBuilder extends RouteBuilder {
  @override
  Route<dynamic> build(RouteSettings settings) {
    final userId = settings.name?.split('/')[2];
    
    if (userId == null) {
      return MaterialPageRoute(
        builder: (context) => const ErrorScreen('Missing user ID'),
        settings: settings,
      );
    }
    
    return MaterialPageRoute(
      builder: (context) => UserScreen(userId: userId),
      settings: settings,
    );
  }
}

// Order route builder (dynamic with multiple parameters)
class OrderBuilder extends RouteBuilder {
  @override
  Route<dynamic> build(RouteSettings settings) {
    // /order/:orderId/item/:itemId
    final parts = settings.name?.split('/') ?? [];
    
    if (parts.length < 5) {
      return MaterialPageRoute(
        builder: (context) => const ErrorScreen('Invalid order route'),
        settings: settings,
      );
    }
    
    final orderId = parts[2];
    final itemId = parts[4];
    
    return MaterialPageRoute(
      builder: (context) => OrderItemScreen(
        orderId: orderId,
        itemId: itemId,
      ),
      settings: settings,
    );
  }
}

// Screen widgets
class HomeScreen extends StatelessWidget {
  const HomeScreen({super.key});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: const Text('Home'),
      ),
      body: ListView(
        children: [
          ListTile(
            title: const Text('About'),
            onTap: () => Navigator.pushNamed(context, '/about'),
          ),
          ListTile(
            title: const Text('Profile'),
            onTap: () => Navigator.pushNamed(context, '/profile', arguments: 'user-123'),
          ),
          ListTile(
            title: const Text('Product 123'),
            onTap: () => Navigator.pushNamed(context, '/product/123'),
          ),
          ListTile(
            title: const Text('Product with Category'),
            onTap: () => Navigator.pushNamed(
              context,
              '/product/456',
              arguments: 'electronics',
            ),
          ),
          ListTile(
            title: const Text('User Alice'),
            onTap: () => Navigator.pushNamed(context, '/user/alice'),
          ),
          ListTile(
            title: const Text('Order Item'),
            onTap: () => Navigator.pushNamed(context, '/order/ORD-001/item/ITEM-123'),
          ),
        ],
      ),
    );
  }
}

class AboutScreen extends StatelessWidget {
  const AboutScreen({super.key});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(title: const Text('About')),
      body: const Center(child: Text('About Screen')),
    );
  }
}

class ProfileScreen extends StatelessWidget {
  final String? userId;
  
  const ProfileScreen({super.key, this.userId});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(title: const Text('Profile')),
      body: Center(
        child: Column(
          mainAxisAlignment: MainAxisAlignment.center,
          children: [
            Text(userId != null ? 'User ID: $userId' : 'No user ID'),
            const SizedBox(height: 20),
            ElevatedButton(
              onPressed: () => Navigator.pop(context),
              child: const Text('Go Back'),
            ),
          ],
        ),
      ),
    );
  }
}

class ProductScreen extends StatelessWidget {
  final String productId;
  final String? category;
  
  const ProductScreen({
    super.key,
    required this.productId,
    this.category,
  });

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(title: Text('Product $productId')),
      body: Center(
        child: Column(
          mainAxisAlignment: MainAxisAlignment.center,
          children: [
            if (category != null) Text('Category: $category'),
            Text('Product ID: $productId'),
            const SizedBox(height: 20),
            ElevatedButton(
              onPressed: () => Navigator.pop(context),
              child: const Text('Go Back'),
            ),
          ],
        ),
      ),
    );
  }
}

class UserScreen extends StatelessWidget {
  final String userId;
  
  const UserScreen({super.key, required this.userId});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(title: Text('User: $userId')),
      body: Center(
        child: ElevatedButton(
          onPressed: () => Navigator.pop(context),
          child: const Text('Go Back'),
        ),
      ),
    );
  }
}

class OrderItemScreen extends StatelessWidget {
  final String orderId;
  final String itemId;
  
  const OrderItemScreen({
    super.key,
    required this.orderId,
    required this.itemId,
  });

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(title: Text('Order $orderId')),
      body: Center(
        child: Column(
          mainAxisAlignment: MainAxisAlignment.center,
          children: [
            Text('Order ID: $orderId'),
            Text('Item ID: $itemId'),
            const SizedBox(height: 20),
            ElevatedButton(
              onPressed: () => Navigator.pop(context),
              child: const Text('Go Back'),
            ),
          ],
        ),
      ),
    );
  }
}

class ErrorScreen extends StatelessWidget {
  final String message;
  
  const ErrorScreen(this.message, {super.key});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(title: const Text('Error')),
      body: Center(
        child: Column(
          mainAxisAlignment: MainAxisAlignment.center,
          children: [
            const Icon(Icons.error, size: 64, color: Colors.red),
            const SizedBox(height: 16),
            Text(message),
            const SizedBox(height: 20),
            ElevatedButton(
              onPressed: () => Navigator.pop(context),
              child: const Text('Go Back'),
            ),
          ],
        ),
      ),
    );
  }
}

class NotFoundScreen extends StatelessWidget {
  final String routeName;
  
  const NotFoundScreen({super.key, required this.routeName});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(title: const Text('Not Found')),
      body: Center(
        child: Column(
          mainAxisAlignment: MainAxisAlignment.center,
          children: [
            const Icon(Icons.error_outline, size: 64, color: Colors.red),
            const SizedBox(height: 16),
            Text('404: $routeName'),
            const SizedBox(height: 20),
            ElevatedButton(
              onPressed: () => Navigator.pop(context),
              child: const Text('Go Back'),
            ),
          ],
        ),
      ),
    );
  }
}
```

**Explanation:**

- **Separate route generation function**: Moving `_generateRoute` to a separate static function makes the code cleaner and more maintainable. It also makes the function easier to test independently.
- **Route map**: A `Map<String, RouteBuilder>` stores static route builders. This provides a centralized registry of all routes, making it easy to add, remove, or modify routes.
- **`RouteBuilder` abstract class**: Defines a consistent interface for all route builders. Each builder implements the `build` method, which takes `RouteSettings` and returns a `Route`. This pattern is extensible and testable.
- **Individual builder classes**: Each route has its own builder class (`HomeBuilder`, `ProductBuilder`, etc.). This encapsulates the logic for building that specific route, including parameter extraction and validation.
- **Error handling in builders**: Each builder validates the route and parameters before building the screen. If validation fails, it returns an `ErrorScreen`. This prevents crashes and provides user-friendly error messages.
- **Complex route parsing**: The `OrderBuilder` handles routes with multiple parameters (`/order/:orderId/item/:itemId`). It checks the path length and extracts multiple parameters by index.
- **Null safety**: The code uses `settings.name?.split('/')` and `?? []` to handle null values safely. This prevents runtime errors when the route name is null or the path is malformed.
- **Extensibility**: To add a new route, you simply:
  1. Create a new builder class extending `RouteBuilder`
  2. Add an entry to the `_routeBuilders` map (for static routes) or add a check in `_generateRoute` (for dynamic routes)
- **Separation of concerns**: This architecture separates:
  - Route registration (the map)
  - Route generation (the `_generateRoute` function)
  - Route building (individual builder classes)
  - Screen rendering (the screen widgets)

---

## **Chapter Summary**

In this chapter, we covered the fundamentals of navigation and routing in Flutter:

### **Key Takeaways:**

1. **Navigator Widget**: The `Navigator` manages a stack of `Route` objects, handling the navigation stack for your app. It's automatically included in `MaterialApp` and `CupertinoApp`.

2. **Route Classes**: Flutter provides several route types:
   - `MaterialPageRoute`: Platform-appropriate transitions
   - `CupertinoPageRoute`: iOS-style transitions
   - `PageRouteBuilder`: Custom transitions with full control
   - `fullscreenDialog`: Modal-style pages

3. **Navigation Methods**:
   - `push()`: Add a route to the stack
   - `pop()`: Remove the top route from the stack
   - `pushReplacement()`: Replace the current route
   - `popUntil()`: Pop multiple routes based on a predicate
   - `pushAndRemoveUntil()`: Push and remove routes

4. **Passing Data**:
   - **Forward direction**: Pass data via constructor parameters
   - **Backward direction**: Return data using `Navigator.pop(context, result)` and capture it with `await Navigator.push()`
   - **Complex objects**: Use data models to encapsulate related data

5. **Named Routes**:
   - Define routes in a `Map<String, WidgetBuilder>` in `MaterialApp`
   - Navigate using `Navigator.pushNamed(context, '/route')`
   - Use `pushReplacementNamed` and `pushNamedAndRemoveUntil` for different behaviors

6. **Named Route Arguments**:
   - Pass arguments via the `arguments` parameter
   - Retrieve arguments using `ModalRoute.of(context)?.settings.arguments`
   - Always check argument types before using them

7. **`onGenerateRoute`**:
   - Provides a callback for dynamic route generation
   - Called whenever a named route is pushed
   - Allows parsing route parameters from the path
   - Return `null` to trigger `onUnknownRoute`

8. **`onUnknownRoute`**:
   - Fallback for undefined routes (404 handler)
   - Called when `onGenerateRoute` returns `null`
   - Show a user-friendly error screen

9. **Route Settings**:
   - Contains the route name and arguments
   - Can be passed to route constructors
   - Essential for deep linking and web support

### **Next Steps:**

Now that you understand the fundamentals of navigation and routing, the next chapter will cover deep linking and Navigation 2.0, including:

- Router and RouteInformationParser
- Declarative routing with Navigator 2.0
- URL-based navigation (web support)
- Deep linking configuration (Android/iOS)
- Universal Links and App Links

---

**End of Chapter 16**

---

# **Next Chapter: Chapter 17 - Deep Linking & Navigation 2.0**

Chapter 17 will explore Flutter's advanced navigation system (Navigator 2.0), which provides a declarative API for managing the navigation stack. You'll learn how to handle deep links, support web URLs, and implement more complex navigation patterns that are impossible with Navigator 1.0.